# REopt CHPbeta testing by calling API locally or staging server

## Initialization

In [1]:
from results_poller import results_plots, reo_optimize, reo_optimize_chpstaging, reo_optimize_localhost
from API_REopt_results import read_scenario_data
import pandas as pd
import numpy as np
import pickle
import json
import requests
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
# For REopt Desktop importing results
from REopt_results import REopt_TempResults

## Load a .pickle file to get previous API response and scenario data 

In [ ]:
save_directory = 'saved_results_6/'
pickle_file = 'DEN_MT-PV_base_hospital'
with open(save_directory + pickle_file + '.pickle', 'rb') as handle:
    api_response_and_scenario_data = pickle.load(handle)
base_api_response = api_response_and_scenario_data['API_response']
base_scenario_data = api_response_and_scenario_data['scenario_data']

#  Load another pickle file

In [ ]:
save_directory = 'saved_results_6/'
pickle_file =  'NYC_MT-PV_base_hospital'
with open(save_directory + pickle_file + '.pickle', 'rb') as handle:
    api_response_and_scenario_data = pickle.load(handle)
api_response = api_response_and_scenario_data['API_response']
scenario_data = api_response_and_scenario_data['scenario_data']

## Scenario Inputs (Post), if wanting to do a new API call

In [2]:
#loads_filepath = "CHP_SanFran_Loads.csv"
#loads_all_dict = pd.read_csv(loads_filepath)
#boiler_fuel_load_mmbtu = list(loads_all_dict['Heat+HW [MMBtu]'])
#electric_load_kw = list(loads_all_dict['Elec Load Minus Chilling [kW]'])
#chiller_electric_load_kw = list(loads_all_dict['Cooling [kW]'])

location = 'DEN'

if location == 'SF':
    ambient_temperature = list(pd.read_csv("SanFran_Tamb_degF.csv", squeeze=True, header = None))
    lat = 37.78
    long = -122.45
    elevation = 0
    nat_gas_price = 7.9 #9.6 commercial
elif location == 'PHX':
    ambient_temperature = list(pd.read_csv("PHX_Tamb_degF.csv", squeeze=True, header = None))
    lat = 33.45
    long = -111.983
    elevation = 1106
    nat_gas_price = 3 # 3 industrial # 6.64 comm (Feb 2020)
elif location == 'CHI':
    ambient_temperature = list(pd.read_csv("CHI_Tamb_degF.csv", squeeze=True, header = None))
    lat = 41.983
    long = -87.917
    elevation = 659
    nat_gas_price = 4.16 #6.7 commercial
elif location == 'NYC':
    ambient_temperature = list(pd.read_csv("NYC_Tamb_degF.csv", squeeze=True, header = None))
    lat = 40.65
    long = -73.8
    elevation = 16
    nat_gas_price = 6.15
elif location == 'DEN':
    ambient_temperature = list(pd.read_csv("Golden_Tamb_degF.csv", squeeze=True, header = None))
    lat = 39.742
    long = -105.179
    elevation = 6000
    nat_gas_price = 5.7
elif location == 'LA':
    ambient_temperature = list(pd.read_csv("LA_Tamb_degF.csv", squeeze=True, header = None))
    lat = 33.933
    long = -118.4
    elevation = 32
    nat_gas_price = 7.9 #9.6 commercial # 7.9 ind
    
#nat_gas_price = 4

post_1 = {"Scenario": {
    "timeout_seconds": 500,
    "optimality_tolerance_techs": 0.05,
    "use_decomposition_model": False,
    "Site": {
    "latitude": lat, "longitude": long,
    "outdoor_air_temp_degF": ambient_temperature,
    "elevation_ft": elevation,  
    #"Financial": {
    #    "om_cost_escalation_pct": 0.01,
    #    "escalation_pct": 0.006,
    #    "offtaker_discount_pct": 0.07,
    #    "analysis_years": 25,
    #    "offtaker_tax_pct": 0.0
    #},   
    "LoadProfile": {
        "doe_reference_name": "Hospital",
        #"doe_reference_name": "LargeHotel",
        #"loads_kw": electric_load_kw
    },
    "LoadProfileBoilerFuel": {
        "doe_reference_name": "Hospital",
        #"doe_reference_name": "LargeHotel",
        #"annual_mmbtu": 12000,
        #"loads_mmbtu_per_hour": boiler_fuel_load_mmbtu,
    },
    "Boiler": {
        "existing_boiler_production_type_steam_or_hw": "hot_water",
        "boiler_efficiency": 0.8
    },
    "LoadProfileChillerElectric": {
        "doe_reference_name": "Hospital",
        #"doe_reference_name": "LargeHotel",
        #"loads_kw": chiller_electric_load_kw
    },
#     "ElectricChiller": {
#         "chiller_cop": 3.5
#     },
    "ElectricTariff": {
        #"urdb_label": '5e1676e95457a3f87673e3b0', # San Fran PG&E
        #"urdb_label": '5cef0a415457a33576f60fe2', # San Fran PG&E - E-19 Medium General Demand TOU Secondary - Comm. - Bundle
        #"urdb_label": '5e1678075457a3d35f73e3b0', # San Fran PG&E - E-20 Maximum demand of 1000KW or more Option R (Secondary) - Comm. - Bundle
        #"urdb_label": '5e611fec5457a3665e019406', # Denver Public Service Co. of CO - 
        "urdb_label": '5a6b921b5457a3c650e4a644', # Denver Public Service Co. of CO - Secondary general low load factor
        #"urdb_label": '5cd200395457a3c62754e9d3', # NYC Edison - General Large TOD Service (NYC)
        #"urdb_label": '5cd2f6935457a3f73d54e9d4', # NYC Edison - Multi dwelleings voluntary TOD sercive
        #"urdb_label": '5caf93355457a35c5f7780e3', # AZ Public Service Co. - Large General Service TOU (E-32 L) Primary
        #"urdb_label": '5caf94185457a326617780e4', # AZ Public Service Co. - Large General Service TOU (E-32 L) Transmission
        #"urdb_label": '5cc76c2a5457a34a0ee327e3', # CHI Commmon. Edison - BES Very large Load Secondary - Comm - Delivery
        #"urdb_label": '5ed698d45457a3a805dd15ae', # LA Southern California Edison Co - Time-Of-Use - General Service - Large: TOU-8, Option D (Under 2 kV)
        #"urdb_label": '5d9aba5d5457a3875061b9f1', # LA Southern California Edison Co - Time-Of-Use - General Service - Large: TOU-8, Option R (under 2kV)
        #"urdb_label": '5e109a4f5457a3e73efa5c5d', # LA Southern California Edison Co - Time-Of-Use - General Service - Large: TOU-8, Option B (under 2kV)
        #"net_metering_limit_kw": 1e6,
        #"wholesale_rate_us_dollars_per_kwh": 0
    },
    "FuelTariff": {
        "existing_boiler_fuel_type": "natural_gas",
        "boiler_fuel_blended_annual_rates_us_dollars_per_mmbtu": nat_gas_price,
        #"boiler_fuel_blended_annual_rates_us_dollars_per_mmbtu": 11.0, # SF (given)
        #"boiler_fuel_blended_annual_rates_us_dollars_per_mmbtu": 7.9, # SF (March 2020) 
        "chp_fuel_type": "natural_gas",
        "chp_fuel_blended_annual_rates_us_dollars_per_mmbtu": nat_gas_price
        #"chp_fuel_blended_annual_rates_us_dollars_per_mmbtu": 11.0 # SF (given)
        #"chp_fuel_blended_annual_rates_us_dollars_per_mmbtu": 7.9 # SF (March 2020)
    },
    "CHP": {
        "prime_mover": "micro_turbine",
        "min_kw": 30,
        #"min_allowable_kw": 30,
        "max_kw": 1000,
        "use_default_derate": True, #True = consider weather, False = ISO only
        "max_derate_factor": 1,
        "derate_start_temp_degF": 72,
        "derate_slope_pct_per_degF": 0.005,
        "elec_effic_full_load": 0.30,
        "elec_effic_half_load": 0.271,
        "thermal_effic_full_load": 0.365,
        "thermal_effic_half_load": 0.351,
        # use private api defaults
        "installed_cost_us_dollars_per_kw": 3200,
        "om_cost_us_dollars_per_kw": 60,
        "om_cost_us_dollars_per_kwh": 0.003,
      },
#    "ColdTES": 
#        "min_gal": 0,
#        "max_gal": 100000,
#         "installed_cost_us_dollars_per_gal": 3,
#         "thermal_decay_rate_fraction": 0.004,
#         "om_cost_us_dollars_per_gal": 0,
#         "internal_efficiency_pct": 0.97,
#     },
#     "HotTES": {
#         "min_gal": 2000,
#         "max_gal": 2000,
#         "installed_cost_us_dollars_per_gal": 3,
#         "thermal_decay_rate_fraction": 0.004,
#         "om_cost_us_dollars_per_gal": 0,
#         "internal_efficiency_pct": 0.97,
#     },
#    "AbsorptionChiller": {
#         "min_ton": 0,
#         "max_ton": 1000,
#         "chiller_cop": 0.7,
#         "installed_cost_us_dollars_per_ton": 500,
#         "om_cost_us_dollars_per_ton": 0.5,
#    },
     "PV": {
        "min_kw": 30,
        "max_kw": 2000,
        #"installed_cost_us_dollars_per_kw": 1500.0,
        #"om_cost_us_dollars_per_kw": 16,
     },
    "Storage": {
        "min_kwh": 0,
        "max_kwh": 0,
        "min_kw": 0,
        "max_kw": 0,
    }
}}}

## Save the post to a .json file

In [ ]:
# Convert python dictionary post into json and save to a .json file
with open('min_inputs_chp_only_SanFran_hospital.json', 'w') as fp:
    json.dump(post_1, fp)

## Or load in a saved .json file for the post

In [ ]:
# Load a json into a python dictionary
with open('POST_chp_sanfran_hospital.json', 'r') as fp:
    post_1 = json.load(fp)

In [ ]:
post_2

## Post and poll API to get a new result, if not loading in from .pickle file
## ...This may take a while!

# Post 1

In [5]:
#api_response = reo_optimize_localhost(post_1)
api_response = reo_optimize_localhost(post_1)

Response OK from http://localhost:8000/v1/job/?format=json&api_key=uaz52dr5KTT5Qs5U72rS70hw3IYeHVEyAaDUFQvo.
Polling http://localhost:8000/v1/job/e61bee32-3c0d-4221-bd55-214ccd4e6fed/results/?api_key=uaz52dr5KTT5Qs5U72rS70hw3IYeHVEyAaDUFQvo for results with interval of 10s...


In [4]:
api_response["outputs"]["Scenario"]["status"]

'optimal'

In [5]:
#base_size_kw = base_api_response['outputs']['Scenario']['Site']['CHP']['size_kw']
#print(base_size_kw)


size_kw = api_response['outputs']['Scenario']['Site']['CHP']['size_kw']
print(size_kw)
size_pv_kw = api_response['outputs']['Scenario']['Site']['PV']['size_kw']
print(size_pv_kw)
size_bess_kw = api_response['outputs']['Scenario']['Site']['Storage']['size_kw']
print(size_bess_kw)
size_bess_kwh = api_response['outputs']['Scenario']['Site']['Storage']['size_kwh']
print(size_bess_kwh)
size_hotTES_gal = api_response['outputs']['Scenario']['Site']['HotTES']['size_gal']
print(size_hotTES_gal)
size_ChillerAbs_ton = api_response['outputs']['Scenario']['Site']['AbsorptionChiller']['size_ton']
print(size_ChillerAbs_ton)

optimality = api_response['outputs']['Scenario']['status']
print(optimality)
time = api_response['outputs']['Scenario']['Profile']['reopt_seconds']
print(time)


#financial 
lcc = api_response['outputs']['Scenario']['Site']['Financial']['lcc_us_dollars']
print(lcc)
lcc_bau = api_response['outputs']['Scenario']['Site']['Financial']['lcc_bau_us_dollars']
print(lcc_bau)
lcc_savings = lcc_bau - lcc
print(lcc_savings)
npv = api_response['outputs']['Scenario']['Site']['Financial']['npv_us_dollars']
print(npv)





940.8932517933254
1680.4569
0.0
0.0
0.0
0.0
optimal
118.58959197998047
10058109.0
18666763.0
8608654.0
8608654.0


# Dispatch code

Data Processing for one scenario

In [ ]:
from scipy.stats import linregress
chp_elec_eff_full_load = api_response['inputs']['Scenario']['Site']['CHP']['elec_effic_full_load']
chp_elec_eff_half_load = api_response['inputs']['Scenario']['Site']['CHP']['elec_effic_half_load']
chp_therm_eff_full_load = api_response['inputs']['Scenario']['Site']['CHP']['thermal_effic_full_load']
chp_therm_eff_half_load = api_response['inputs']['Scenario']['Site']['CHP']['thermal_effic_half_load']
chp_size_kwe = api_response['outputs']['Scenario']['Site']['CHP']['size_kw']
CHP_fuel_burn_full_load = chp_size_kwe / chp_elec_eff_full_load * 3412/1e6 #[MMBtu/hr]
CHP_fuel_burn_half_load = 0.5*chp_size_kwe / chp_elec_eff_half_load * 3412/1e6
slope, intercept, rvalue, pvalue, stderr = linregress([0.5*chp_size_kwe, chp_size_kwe], [CHP_fuel_burn_half_load, CHP_fuel_burn_full_load])
CHP_elec_slope = slope
CHP_elec_intercept = intercept
CHP_max_heat_full_load = CHP_fuel_burn_full_load * chp_therm_eff_full_load #[MMBtu/hr]
CHP_max_heat_half_load = CHP_fuel_burn_half_load * chp_therm_eff_half_load 
slope_tp, intercept_tp, rvalue, pvalue, stderr = linregress([0.5*chp_size_kwe, chp_size_kwe], [CHP_max_heat_half_load, CHP_max_heat_full_load])
CHP_therm_slope = slope_tp
CHP_therm_intercept = intercept_tp
print(CHP_elec_slope)
print(CHP_elec_intercept)
print(CHP_therm_slope)
print(CHP_therm_intercept)


load_year = str(api_response['inputs']['Scenario']['Site']['LoadProfile']['year'])
dti = pd.date_range( load_year +'-01-01', periods=8760, freq='H')
bau_grid_load_less_chiller_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']
                                               ['LoadProfile']['year_one_electric_load_series_kw'], index = dti)
bau_boiler_load_fuel_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']
                                         ['LoadProfileBoilerFuel']['year_one_boiler_fuel_load_series_mmbtu_per_hr_bau'], index = dti)
bau_chiller_load_elec_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']
                                          ['LoadProfileChillerElectric']['year_one_chiller_electric_load_series_kw'], index = dti)
boiler_eff = api_response['inputs']['Scenario']['Site']['Boiler']['boiler_efficiency']

bau_grid_load_hrly = bau_grid_load_less_chiller_hrly + bau_chiller_load_elec_hrly
bau_grid_load_peak = np.max(bau_grid_load_hrly)
bau_grid_load_mean = np.mean(bau_grid_load_hrly)
bau_grid_load_total = np.sum(bau_grid_load_hrly)
bau_therm_load_peak = np.max(bau_boiler_load_fuel_hrly)*boiler_eff #converted units of fuel to units of heat
bau_therm_load_mean = np.mean(bau_boiler_load_fuel_hrly)*boiler_eff
bau_therm_load_total = np.sum(bau_boiler_load_fuel_hrly)*boiler_eff
chp_therm_to_elec_full_load = chp_therm_eff_full_load/chp_elec_eff_full_load

bau_therm_load_min = np.min(bau_boiler_load_fuel_hrly)*boiler_eff
bau_grid_load_min = np.min(bau_grid_load_hrly)
bau_chiller_thermal_load_max = (np.max(bau_chiller_load_elec_hrly) * 3.5) * 0.2843451361
bau_chiller_thermal_load_mean = (np.mean(bau_chiller_load_elec_hrly) * 3.5) * 0.2843451361
bau_chiller_thermal_load_min = (np.min(bau_chiller_load_elec_hrly) * 3.5) * 0.2843451361
print("Heating min load =", '%.2f' %bau_therm_load_min, "MMBtu")
print("Elec min load =", '%.2f' %bau_grid_load_min, "kW")
print("Cooling max load =", '%.2f' %bau_chiller_thermal_load_max, "ton")
print("Cooling mean load =", '%.2f' %bau_chiller_thermal_load_mean, "ton")
print("Cooling min load =", '%.2f' %bau_chiller_thermal_load_min, "ton")




chp_elec_total_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_electric_production_series_kw'], index = dti) #total CHP elec production (hourly)
chp_elec_to_load_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_to_load_series_kw'], index = dti) #CHP elec produciton to load (hourly)
chp_elec_to_grid_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_to_grid_series_kw'], index = dti) #CHP elec production to grid (hourly)

grid_load_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']
                                               ['LoadProfile']['year_one_electric_load_series_kw'], index = dti)







#-----chris-calc electric efficiency----
slope_ee, intercept_ee, rvalue, pvalue, stderr = linregress([0.5*chp_size_kwe, chp_size_kwe], [100*chp_elec_eff_half_load, 100*chp_elec_eff_full_load])
print("Electric Efficiency Slope = ", '%.4f' %slope_ee)
print("Electric Efficiency Intercept = ", '%.1f' %intercept_ee)        
year_one_elec_eff_series_percent = slope_ee*np.array(chp_elec_total_hrly.values)+ intercept_ee

# fuel burn
slope_fb, intercept_fb, rvalue, pvalue, stderr = linregress([0.5*chp_size_kwe, chp_size_kwe], [CHP_fuel_burn_half_load, CHP_fuel_burn_full_load])
corr = 1 # 1 = MT corr , 2 = ICE corr, else = base
cf_fb = np.ones(8760)
t_ref = 77  # [deg F]
if corr == 1:
    T_ISO = 59
    for i in range(8760):
        cf_fb[i] = 0.9529*np.exp(0.0008249*ambient_temperature[i]) - (7.496*(10**(-6)))*np.exp(0.06726*ambient_temperature[i])
if corr == 2:
    T_ISO = 77
    for i in range(8760):
        if ambient_temperature[i] <= t_ref:
                 cf_fb[i] = 1 + 0.0003*(ambient_temperature[i] - t_ref)
        else:
                 cf_fb[i] = 1
else:
    T_ISO = 59
    
chp_elec_total_hrly_a = np.array(chp_elec_total_hrly)
year_one_fuel_burn_series = np.ones(8760)
year_one_elec_eff_series_percent_corr = np.ones(8760)
for i in range(8760):
	year_one_fuel_burn_series[i] = cf_fb[i]*(slope_fb*chp_elec_total_hrly_a[i] + intercept_fb)
for i in range(8760):    
    year_one_elec_eff_series_percent_corr[i] = 100*(chp_elec_total_hrly_a[i] / (year_one_fuel_burn_series[i]* (1e6/3412)))    


#thermal production
year_one_thermal_prod_series_corr = np.ones(8760)
cf_tp = np.ones(8760)
if corr == 1:
    for i in range(8760):
        cf_tp[i] = 0.005852*ambient_temperature[i] + 0.6615
if corr == 2:
    for i in range(8760):
        if ambient_temperature[i] <= t_ref:
                 cf_tp[i] = 1 + 0.001*(ambient_temperature[i] - t_ref)
        else:
                 cf_tp[i] = 1
for i in range(8760):
	year_one_thermal_prod_series_corr[i] = cf_tp[i]*(slope_tp*chp_elec_total_hrly_a[i] + intercept_tp)                
                                                                                               

hours = np.linspace(1,8760,8760)
s = 8000 #start hour mark
e = 8050 #end hour mark
plt.figure()
plt.plot(hours[s:e], year_one_elec_eff_series_percent[s:e], color='green') #linestyle='none',marker='o') 
#------end----

electric_demand = np.array(api_response['outputs']['Scenario']['Site']['LoadProfile']['year_one_electric_load_series_kw'])
chp_power = np.array(api_response['outputs']['Scenario']['Site']['CHP']['year_one_electric_production_series_kw'])
chp_power_to_load = np.array(api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_to_load_series_kw'])
chp_power_to_grid = np.array(api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_to_grid_series_kw'])
one = np.zeros(8760)

save = 2 # 1 = save , 2 = do not save
mode = 1 #CHP+PV = 1 , CHP = 2
if mode == 1:
#-----chris-plot electric load profiles----
    pv_elec_total_hrly = np.array(api_response['outputs']['Scenario']['Site']['PV']
                                   ['year_one_power_production_series_kw']) #PV hourly total elec production
    pv_elec_to_load_hrly = np.array(api_response['outputs']['Scenario']['Site']['PV']
                                   ['year_one_to_load_series_kw']) #PV hourly elec production to load
    pv_elec_to_grid_hrly = np.array(api_response['outputs']['Scenario']['Site']['PV']
                                   ['year_one_to_grid_series_kw']) #PV hourly elec production to grid
    grid_elec_to_load_hrly = electric_demand - chp_power_to_load - pv_elec_to_load_hrly
    pv_plus_chp_to_load_hrly = chp_power_to_load + pv_elec_to_load_hrly

#show load profile and distribution amongst DG techs
    tolerance = 0.01 #sometimes tech power output just barely exceeds the demand and messes up plotting
    plt.figure() 
    line1, = plt.plot(hours[s:e], electric_demand[s:e],'k', label = "Demand")
    #line2, = plt.plot(hours[s:e], chp_power_to_load[s:e], 'b',  label = "CHP")
    #line3, = plt.plot(hours[s:e], pv_plus_chp_to_load_hrly[s:e], 'r', label = "PV")
    #line4, = plt.plot(hours[s:e], grid_elec_to_load_hrly[s:e], 'r', label = "Grid")
    fill3 = plt.fill_between(hours[s:e], pv_plus_chp_to_load_hrly[s:e], electric_demand[s:e], \
                     where=   electric_demand[s:e] + tolerance >= pv_plus_chp_to_load_hrly[s:e] , facecolor = 'green', alpha = 0.5, label = 'Grid' )
    fill2 = plt.fill_between(hours[s:e], pv_plus_chp_to_load_hrly[s:e], chp_power_to_load[s:e], \
                     where=   pv_plus_chp_to_load_hrly[s:e] >= chp_power_to_load[s:e] , facecolor = 'red', alpha = 0.5, label = 'PV' )
    fill1 = plt.fill_between(hours[s:e], chp_power_to_load[s:e], one[s:e], \
                     where=  chp_power_to_load[s:e] >= one[s:e] , facecolor = 'blue', alpha = 0.5, label = 'CHP' )
    plt.xlabel('Hours')
    plt.ylabel('Electric Power [kWe]')
    plt.legend(handles = [fill1, fill2, fill3, line1])
    if save == 1:
        plt.savefig('figures/CHI/CHI_ICE-PV_corr_e_profile_summer.png', dpi = 300)
#show DG power flow to grid versus load (ignore battery)
    plt.figure() 
    line1, = plt.plot(hours[s:e], chp_elec_to_load_hrly[s:e], 'b',  label = "CHP to load")
    line2, = plt.plot(hours[s:e], chp_elec_to_grid_hrly[s:e], 'b--',  label = "CHP to grid")
    line3, = plt.plot(hours[s:e], pv_elec_to_load_hrly[s:e], 'g', label = "PV to load")
    line4, = plt.plot(hours[s:e], pv_elec_to_grid_hrly[s:e], 'g--', label = "PV to grid")
    plt.legend(handles = [line1, line2, line3, line4])
    plt.xlabel('Hours')
    plt.ylabel('Electric Power [kWe]')
    
    #calculate CHP distributions
    chp_power_to_load_plus_grid = chp_power_to_load + chp_power_to_grid
    plt.figure() 
    #line1, = plt.plot(hours[s:e], chp_power_to_load[s:e], 'b',  label = "CHP to load")
    #line2, = plt.plot(hours[s:e], chp_power_to_load_plus_grid[s:e], 'g',  label = "CHP to grid")
    #line3, = plt.plot(hours[s:e], chp_elec_to_bess_hrly[s:e], 'b',  label = "CHP to BESS")
    line4, = plt.plot(hours[s:e], chp_power[s:e], 'k',  label = "Total CHP", linewidth=2)
    
    fill1 = plt.fill_between(hours[s:e], chp_power_to_load[s:e], one[s:e], \
                     where=  chp_power_to_load[s:e] >= one[s:e] , facecolor = 'blue', alpha = 0.5, label = 'CHP to Load' )
    fill2 = plt.fill_between(hours[s:e], chp_power_to_load[s:e], chp_power[s:e], \
                     where=  chp_power[s:e] >= chp_power_to_load[s:e] , facecolor = 'green', alpha = 0.5, label = 'CHP to Grid' )
    plt.xlabel('Hours')
    plt.ylabel('Electric Power [kWe]')
    plt.legend(handles = [fill1, fill2, line4])
    #SAVE
    #plt.savefig('figures_5/SF/SF_ICE-PV-BESS_base_CHPprofile_summer.png', dpi = 300)
    
    
    #show PV power distribution
    pv_elec_to_load_plus_grid_hrly = pv_elec_to_load_hrly + pv_elec_to_grid_hrly
    plt.figure()
    #line1, = plt.plot(hours[s:e], pv_elec_to_load_hrly[s:e], 'b', label = "PV to load")
    #line2, = plt.plot(hours[s:e], pv_elec_to_load_plus_grid_hrly[s:e], 'g', label = "PV to Grid")
    #line3, = plt.plot(hours[s:e], pv_elec_to_battery_hrly[s:e], 'g--', label = "PV to BESS")
    line4, = plt.plot(hours[s:e], pv_elec_total_hrly[s:e], 'k',  label = "Total PV", linewidth=2)
    fill1 = plt.fill_between(hours[s:e], pv_elec_to_load_hrly[s:e], one[s:e], \
                     where=  pv_elec_to_load_hrly[s:e] >= one[s:e] , facecolor = 'blue', alpha = 0.5, label = 'PV to Load' )
    fill2 = plt.fill_between(hours[s:e], pv_elec_to_load_hrly[s:e], pv_elec_total_hrly[s:e], \
                     where=  pv_elec_total_hrly[s:e] >= pv_elec_to_load_hrly[s:e] , facecolor = 'green', alpha = 0.5, label = 'PV to Grid' )

    plt.legend(handles = [fill1, fill2, line4])
    plt.xlabel('Hours')
    plt.ylabel('Electric Power [kWe]')
else:
    grid_elec_to_load_hrly = electric_demand - chp_power
    #show load profile and distribution amongst DG techs
    plt.figure() 
    line1, = plt.plot(hours[s:e], electric_demand[s:e],'k', label = "Demand")
    line2, = plt.plot(hours[s:e], chp_power[s:e], 'b',  label = "CHP")
    #line3, = plt.plot(hours[s:e], grid_elec_to_load_hrly[s:e], 'y', label = "Grid")
    fill2 = plt.fill_between(hours[s:e], chp_power[s:e], electric_demand[s:e], \
                     where=   electric_demand[s:e] >= chp_power[s:e] , facecolor = 'green', alpha = 0.5, label = 'Grid' )
    fill1 = plt.fill_between(hours[s:e], chp_power[s:e], one[s:e], \
                     where=  chp_power[s:e] >= one[s:e] , facecolor = 'blue', alpha = 0.5, label = 'CHP' )
    plt.legend(handles = [fill1, fill2, line2])
    plt.xlabel('Hours')
    plt.ylabel('Electric Power [kWe]')
    #grid export?
    plt.figure() 
    line1, = plt.plot(hours[s:e], chp_elec_to_load_hrly[s:e], 'b',  label = "CHP to load")
    line2, = plt.plot(hours[s:e], chp_elec_to_grid_hrly[s:e], 'b--',  label = "CHP to grid")
    plt.legend(handles = [line1, line2])
    plt.xlabel('Hours')
    plt.ylabel('Electric Power [kWe]')
    

    
    
    

chp_therm_to_load_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                      ['year_one_thermal_to_load_series_mmbtu_per_hour'], index = dti)
chp_therm_to_TES_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                     ['year_one_thermal_to_tes_series_mmbtu_per_hour'], index = dti)
chp_therm_total_hrly = chp_therm_to_load_hrly + chp_therm_to_TES_hrly
chp_fuel_annual = api_response['outputs']['Scenario']['Site']['CHP']['year_one_fuel_used_mmbtu']
chp_elec_effic = np.sum(chp_elec_total_hrly)/(chp_fuel_annual * 1E6/3412.0)
chp_total_effic = (np.sum(chp_elec_total_hrly) + np.sum(chp_therm_total_hrly)* 1E6/3412.0)/(chp_fuel_annual * 1E6/3412.0)
chp_elec_cf = np.sum(chp_elec_total_hrly)/(chp_size_kwe*8760)
chp_avg_load_when_on = chp_elec_total_hrly[chp_elec_total_hrly>1].mean()
boiler_fuel_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['Boiler']
                                ['year_one_boiler_fuel_consumption_series_mmbtu_per_hr'], index = dti)
boiler_fuel_annual = boiler_fuel_hrly.resample('Y').sum()









#-----chris-plot thermal load profiles----
boiler_therm_total_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['Boiler']
                                     ['year_one_boiler_thermal_production_series_mmbtu_per_hr'], index = dti)
boiler_fuel_hrly_bau = pd.DataFrame(api_response['outputs']['Scenario']['Site']['LoadProfileBoilerFuel']
                                     ['year_one_boiler_fuel_load_series_mmbtu_per_hr_bau'], index = dti)
therm_load_hrly = boiler_fuel_hrly_bau*boiler_eff

chp_tp = np.array(api_response['outputs']['Scenario']['Site']['CHP']['year_one_thermal_to_load_series_mmbtu_per_hour'])
boiler_tp = np.array(api_response['outputs']['Scenario']['Site']['Boiler']['year_one_boiler_thermal_production_series_mmbtu_per_hr'])
#thermal_demand = np.array(api_response['outputs']['Scenario']['Site']['LoadProfileBoilerFuel']
#                          ['year_one_boiler_fuel_load_series_mmbtu_per_hr_bau'])/boiler_eff
thermal_demand = chp_tp+boiler_tp

#print(np.transpose(chp_therm_total_hrly.values[s:e])
#HotTES_to_load_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['HotTES']
#                                     ['year_one_thermal_from_hot_tes_series_mmbtu_per_hr'], index = dti)
plt.figure()
line1, = plt.plot(hours[s:e], chp_tp[s:e], 'b',  label = "CHP")
#line2, = plt.plot(hours[s:e], boiler_tp[s:e], 'r',  label = "Boiler")
#line3, = plt.plot(hours[s:e], HotTES_to_load_hrly[s:e], 'y',  label = "TES")
line4, = plt.plot(hours[s:e], thermal_demand[s:e], 'k',  label = "Demand") #really"Total"

fill2 = plt.fill_between(hours[s:e], chp_tp[s:e],thermal_demand[s:e], \
                     where=   thermal_demand[s:e] >= chp_tp[s:e] , facecolor = 'green', alpha = 0.5, label = 'Boiler' )
fill1 = plt.fill_between(hours[s:e], chp_tp[s:e], one[s:e], \
                     where=  chp_tp[s:e] >= one[s:e] , facecolor = 'blue', alpha = 0.5, label = 'CHP' )
plt.xlabel('Hours')
plt.ylabel('Thermal Production [MMBtu/hr]')
plt.legend(handles = [fill1, fill2, line4])
if save == 1:
    plt.savefig('figures/CHI/CHI_ICE-PV_corr_tp_profile_summer.png', dpi = 300)



#check CF calcs
plt.figure() 
line2, = plt.plot(hours[s:e], chp_therm_to_load_hrly[s:e], 'g',  label = "Dispatched") #see if sizes are different
line3, = plt.plot(hours[s:e], CHP_max_heat_full_load*np.ones(e-s), 'b',  label = "Rated")
line4, = plt.plot(hours[s:e], year_one_thermal_prod_series_corr[s:e], 'r',  label = "Produced")
plt.legend(handles = [line3, line2, line4])
plt.xlabel('Hours')
plt.ylabel('Thermal Production [MMBtu/hr]')
#plt.ylim([0, 1])
#plt.savefig('figures/CHI/CHI_ICE-PV_corr_tp_profile.png', dpi = 300)

plt.figure() 
line2, = plt.plot(hours[s:e], T_ISO*np.ones(e-s), 'b',  label = "ISO") #see if sizes are different
line3, = plt.plot(hours[s:e], ambient_temperature[s:e], 'g',  label = "TMY3")
plt.legend(handles = [line2, line3])
plt.xlabel('Hours')
plt.ylabel('Ambient Temperature [F]')
#plt.savefig('SF_MT_Tamb.png', dpi = 300)
plt.figure() 
line2, = plt.plot(hours[s:e], chp_size_kwe*np.ones(e-s), 'b',  label = "Rated") #see if sizes are different
line3, = plt.plot(hours[s:e], chp_elec_to_load_hrly[s:e], 'g',  label = "Dispatched")
plt.legend(handles = [line2, line3])
plt.xlabel('Hours')
plt.ylabel('Electric Power [kWe]')
#plt.savefig('figures/PHX/PHX_ICE-PV_corr_e_profile.png', dpi = 300)
plt.figure() 
line2, = plt.plot(hours[s:e], chp_elec_eff_full_load*100*np.ones(e-s), 'b',  label = "Rated") #see uf sizes are different
line3, = plt.plot(hours[s:e], year_one_elec_eff_series_percent_corr[s:e], 'g',  label = "Dispatched")
plt.legend(handles = [line2, line3])
plt.xlabel('Hours')
plt.ylabel('Electric Efficiency [%]')
#plt.ylim([36,38])
#plt.savefig('figures/CHI/CHI_ICE-PV_corr_ee_profile.png', dpi = 300)

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Times New Roman']






#chp vs boiler fuel consumption
year_one_chp_fuel = api_response['outputs']['Scenario']['Site']['CHP']['year_one_fuel_used_mmbtu']
print("Annual CHP Fuel Use =", '%.3f' %year_one_chp_fuel)
year_one_boiler_fuel = api_response['outputs']['Scenario']['Site']['Boiler']['year_one_boiler_fuel_consumption_mmbtu']
print("Annual Boiler Fuel Use =", '%.3f' %year_one_boiler_fuel)



#chp_annual_runhrs = np.count_nonzero(chp_elec_total_hourly)
chp_annual_runhrs = np.sum(chp_elec_total_hrly>1) #alternative to np.count_nonzero to eliminate very small numbers. Sums the number of TRUE
#chp_annual_runhrs = chp_elec_total_hourly[chp_elec_total_hourly>1].count() #alternative to np.count_nonzero to eliminate very small numbers
chp_elec_cf_when_on = chp_avg_load_when_on/chp_size_kwe
chp_annual_heatinghrs = np.sum(chp_therm_total_hrly>0.01) #alternative to np.count_nonzero to eliminate very small numbers
chp_capacity_elec_to_load_peak = chp_size_kwe/bau_grid_load_peak
chp_capacity_elec_to_load_mean = chp_size_kwe/bau_grid_load_mean
chp_capacity_therm_to_load_peak = chp_size_kwe*chp_therm_to_elec_full_load/(bau_therm_load_peak*1e6/3412)
chp_capacity_therm_to_load_mean = chp_size_kwe*chp_therm_to_elec_full_load/(bau_therm_load_mean*1e6/3412)

load_hours_less_than_chp_capacity = np.sum(bau_grid_load_hrly < chp_size_kwe)

#find base load of bau_grid_load_hrly. Using value with 85% exceedance as described in ASHRAE 'Combined Heat and Power Design Guide' 
base_load_site_elec = bau_grid_load_hrly.sort_values(0, ascending = False).iloc[int(8760*.85)-1]
base_load_site_therm = bau_boiler_load_fuel_hrly.sort_values(0, ascending = False).iloc[int(8760*.85)-1]/boiler_eff
chp_capacity_elec_to_base_load = chp_size_kwe/base_load_site_elec
chp_capacity_therm_to_base_load = chp_size_kwe*chp_therm_to_elec_full_load/(base_load_site_therm*1e6/3412)
chp_gen_elec_to_load = chp_elec_total_hrly.resample('Y').sum()/bau_grid_load_hrly.resample('Y').sum()
chp_gen_therm_to_load = chp_therm_total_hrly.resample('Y').sum()/(bau_boiler_load_fuel_hrly.resample('Y').sum()*boiler_eff)
#net_fuel_consumption_change = 

print("Annual CHP Electric Efficiency =", '%.3f' %chp_elec_effic)
print("Annual CHP Total Efficiency =", '%.3f' %chp_total_effic)
print("Annual CHP Elec Capacity Factor =", '%.2f' %chp_elec_cf)
print("Annual CHP Elec Capacity Factor when running =", '%.2f' %chp_elec_cf_when_on)
print("Annual CHP run hours =", '%.0f' %chp_annual_runhrs, "(fraction",'%.2f'%(chp_annual_runhrs/8760), ")")
print("Annual CHP hours of useful heat =", '%.0f' %chp_annual_heatinghrs, "(fraction",'%.2f'%(chp_annual_heatinghrs/8760), ")")
print("Annual CHP mean power output when on =", '%.0f' %chp_avg_load_when_on, "kW")
print("Elec peak load =", '%.0f' %bau_grid_load_peak, "kW")
print("Elec mean load =", '%.0f' %bau_grid_load_mean, "kW")
print("Elec base load =", '%.0f' %base_load_site_elec, "kW (85% exceedance value)")
print("Heating peak load =", '%.2f' %bau_therm_load_peak, "MMBtu")
print("Heating mean load =", '%.2f' %bau_therm_load_mean, "MMBtu")
print("Heating base load =", '%.2f' %base_load_site_therm, "MMBtu (85% exceedance value)")
print("Ratio of CHP elec capacity to peak elec load =", '%.3f' %chp_capacity_elec_to_load_peak)
print("Ratio of CHP elec capacity to mean elec load =", '%.3f' %chp_capacity_elec_to_load_mean)
print("Ratio of CHP elec capacity to base elec load =", '%.2f' %chp_capacity_elec_to_base_load)
print("Ratio of CHP therm capacity to peak heat load =", '%.3f' %chp_capacity_therm_to_load_peak)
print("Ratio of CHP therm capacity to mean heat load =", '%.3f' %chp_capacity_therm_to_load_mean)
print("Ratio of CHP therm capacity to base heat load =", '%.3f' %chp_capacity_therm_to_base_load)
print("Useful heat recovery ratio =", '%.3f' )
print("Annual fraction of site elec load served by CHP =", '%.3f' %chp_gen_elec_to_load.iloc[0][0])
print("Annual fraction of site thermal load served by CHP =", '%.3f' %chp_gen_therm_to_load.iloc[0][0])

#checks #there is a tolerance where tech power is +0.001 above electric demand
#print(electric_demand[5095:5098])
#print(grid_elec_to_load_hrly[5095:5098])
#print(pv_plus_chp_to_load_hrly[5095:5098])
#print(electric_demand[5095:5098] - (pv_plus_chp_to_load_hrly[5095:5098]+grid_elec_to_load_hrly[5095:5098]))
#print(electric_demand[5095:5098] - pv_plus_chp_to_load_hrly[5095:5098])


# # thermal grade calculations

Data Processing to compare 2 scenarios

In [ ]:
from scipy.stats import linregress
chp_elec_eff_full_load = api_response['inputs']['Scenario']['Site']['CHP']['elec_effic_full_load']
chp_elec_eff_half_load = api_response['inputs']['Scenario']['Site']['CHP']['elec_effic_half_load']
chp_therm_eff_full_load = api_response['inputs']['Scenario']['Site']['CHP']['thermal_effic_full_load']
chp_therm_eff_half_load = api_response['inputs']['Scenario']['Site']['CHP']['thermal_effic_half_load']
chp_size_kwe = api_response['outputs']['Scenario']['Site']['CHP']['size_kw']
CHP_fuel_burn_full_load = chp_size_kwe / chp_elec_eff_full_load * 3412/1e6 #[MMBtu/hr]
CHP_fuel_burn_half_load = 0.5*chp_size_kwe / chp_elec_eff_half_load * 3412/1e6
slope_fb, intercept_fb, rvalue, pvalue, stderr = linregress([0.5*chp_size_kwe, chp_size_kwe], [CHP_fuel_burn_half_load, CHP_fuel_burn_full_load])
CHP_elec_slope = slope_fb
CHP_elec_intercept = intercept_fb
CHP_max_heat_full_load = CHP_fuel_burn_full_load * chp_therm_eff_full_load #[MMBtu/hr]
CHP_max_heat_half_load = CHP_fuel_burn_half_load * chp_therm_eff_half_load 
slope_tp, intercept_tp, rvalue, pvalue, stderr = linregress([0.5*chp_size_kwe, chp_size_kwe], [CHP_max_heat_half_load, CHP_max_heat_full_load])
CHP_therm_slope = slope_tp
CHP_therm_intercept = intercept_tp
print(CHP_elec_slope)
print(CHP_elec_intercept)
print(CHP_therm_slope)
print(CHP_therm_intercept)


load_year = str(api_response['inputs']['Scenario']['Site']['LoadProfile']['year'])
dti = pd.date_range( load_year +'-01-01', periods=8760, freq='H')
bau_grid_load_less_chiller_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']
                                               ['LoadProfile']['year_one_electric_load_series_kw'], index = dti)
bau_boiler_load_fuel_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']
                                         ['LoadProfileBoilerFuel']['year_one_boiler_fuel_load_series_mmbtu_per_hr_bau'], index = dti)
bau_chiller_load_elec_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']
                                          ['LoadProfileChillerElectric']['year_one_chiller_electric_load_series_kw'], index = dti)
boiler_eff = api_response['inputs']['Scenario']['Site']['Boiler']['boiler_efficiency']

bau_grid_load_hrly = bau_grid_load_less_chiller_hrly + bau_chiller_load_elec_hrly
bau_grid_load_peak = np.max(bau_grid_load_hrly)
bau_grid_load_mean = np.mean(bau_grid_load_hrly)
bau_grid_load_total = np.sum(bau_grid_load_hrly)
bau_therm_load_peak = np.max(bau_boiler_load_fuel_hrly)*boiler_eff #converted units of fuel to units of heat
bau_therm_load_mean = np.mean(bau_boiler_load_fuel_hrly)*boiler_eff
bau_therm_load_total = np.sum(bau_boiler_load_fuel_hrly)*boiler_eff
chp_therm_to_elec_full_load = chp_therm_eff_full_load/chp_elec_eff_full_load


chp_elec_total_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_electric_production_series_kw'], index = dti) #total CHP elec production (hourly)
chp_elec_to_load_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_to_load_series_kw'], index = dti) #CHP elec produciton to load (hourly)
chp_elec_to_grid_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_to_grid_series_kw'], index = dti) #CHP elec production to grid (hourly)



#-----chris-calc electric efficiency----
slope_ee, intercept_ee, rvalue, pvalue, stderr = linregress([0.5*chp_size_kwe, chp_size_kwe], [100*chp_elec_eff_half_load, 100*chp_elec_eff_full_load])
print("Electric Efficiency Slope = ", '%.4f' %slope_ee)
print("Electric Efficiency Intercept = ", '%.1f' %intercept_ee)        
year_one_elec_eff_series_percent = slope_ee*np.array(chp_elec_total_hrly.values)+ intercept_ee

# fuel burn
slope_fb, intercept_fb, rvalue, pvalue, stderr = linregress([0.5*chp_size_kwe, chp_size_kwe], [CHP_fuel_burn_half_load, CHP_fuel_burn_full_load])
corr = 1 # 1 = MT corr , 2 = ICE corr, else = base
cf_fb = np.ones(8760)
t_ref = 77  # [deg F]
if corr == 1:
    T_ISO = 59
    for i in range(8760):
        cf_fb[i] = 0.9529*np.exp(0.0008249*ambient_temperature[i]) - (7.496*(10**(-6)))*np.exp(0.06726*ambient_temperature[i])
if corr == 2:
    T_ISO = 77
    for i in range(8760):
        if ambient_temperature[i] <= t_ref:
                 cf_fb[i] = 1 + 0.0003*(ambient_temperature[i] - t_ref)
        else:
                 cf_fb[i] = 1
else:
    T_ISO = 59
    
chp_elec_total_hrly_a = np.array(chp_elec_total_hrly)
year_one_fuel_burn_series = np.ones(8760)
year_one_elec_eff_series_percent_corr = np.ones(8760)
for i in range(8760):
	year_one_fuel_burn_series[i] = cf_fb[i]*(slope_fb*chp_elec_total_hrly_a[i] + intercept_fb)
for i in range(8760):    
    year_one_elec_eff_series_percent_corr[i] = 100*(chp_elec_total_hrly_a[i] / (year_one_fuel_burn_series[i]* (1e6/3412)))    


#thermal production
year_one_thermal_prod_series_corr = np.ones(8760)
cf_tp = np.ones(8760)
if corr == 1:
    for i in range(8760):
        cf_tp[i] = 0.005852*ambient_temperature[i] + 0.6615
if corr == 2:
    for i in range(8760):
        if ambient_temperature[i] <= t_ref:
                 cf_tp[i] = 1 + 0.001*(ambient_temperature[i] - t_ref)
        else:
                 cf_tp[i] = 1
for i in range(8760):
	year_one_thermal_prod_series_corr[i] = cf_tp[i]*(slope_tp*chp_elec_total_hrly_a[i] + intercept_tp)  


#base electric efficiency
base_chp_elec_total_hrly = pd.DataFrame(base_api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_electric_production_series_kw'], index = dti)
base_chp_elec_eff_full_load = base_api_response['inputs']['Scenario']['Site']['CHP']['elec_effic_full_load']
base_chp_elec_eff_half_load = base_api_response['inputs']['Scenario']['Site']['CHP']['elec_effic_half_load']
base_chp_size_kwe = base_api_response['outputs']['Scenario']['Site']['CHP']['size_kw']
base_slope_ee, base_intercept_ee, base_rvalue, base_pvalue, base_stderr = linregress([0.5*base_chp_size_kwe, base_chp_size_kwe], [100*base_chp_elec_eff_half_load, 100*base_chp_elec_eff_full_load])
print("Electric Efficiency Slope = ", '%.4f' %slope_ee)
print("Electric Efficiency Intercept = ", '%.1f' %intercept_ee)
base_year_one_elec_eff_series_percent = base_slope_ee*base_chp_elec_total_hrly.values+ base_intercept_ee



hours = np.linspace(1,8760,8760)
s = 4000 #start hour mark
e = 4100 #end hour mark
#------end----

electric_demand = np.array(api_response['outputs']['Scenario']['Site']['LoadProfile']['year_one_electric_load_series_kw'])
chp_power = np.array(api_response['outputs']['Scenario']['Site']['CHP']['year_one_electric_production_series_kw'])
chp_power_to_load = np.array(api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_to_load_series_kw'])
one = np.zeros(8760)

mode = 1 #CHP+PV = 1 , CHP = 2, CHP+PV+BESS = 3
if mode == 1:
#-----chris-plot electric load profiles----
    pv_elec_total_hrly = np.array(api_response['outputs']['Scenario']['Site']['PV']
                                   ['year_one_power_production_series_kw']) #PV hourly total elec production
    pv_elec_to_load_hrly = np.array(api_response['outputs']['Scenario']['Site']['PV']
                                   ['year_one_to_load_series_kw']) #PV hourly elec production to load
    pv_elec_to_grid_hrly = np.array(api_response['outputs']['Scenario']['Site']['PV']
                                   ['year_one_to_grid_series_kw']) #PV hourly elec production to grid
    grid_elec_to_load_hrly = electric_demand - chp_power_to_load - pv_elec_to_load_hrly
    pv_plus_chp_to_load_hrly = chp_power_to_load + pv_elec_to_load_hrly

#show load profile and distribution amongst DG techs
    plt.figure() 
    line1, = plt.plot(hours[s:e], electric_demand[s:e],'k', label = "Total")
    #line2, = plt.plot(hours[s:e], chp_power_to_load[s:e], 'b',  label = "CHP")
    #line3, = plt.plot(hours[s:e], pv_plus_chp_to_load_hrly[s:e], 'r', label = "PV")
    #line4, = plt.plot(hours[s:e], grid_elec_to_load_hrly[s:e], 'r', label = "Grid")
    #plt.legend(handles = [line1, line2, line3])
    plt.fill_between(hours[s:e], pv_plus_chp_to_load_hrly[s:e], electric_demand[s:e], \
                     where=   electric_demand[s:e] + 0.01 >= pv_plus_chp_to_load_hrly[s:e] , facecolor = 'green', alpha = 0.5 )
    plt.fill_between(hours[s:e], pv_plus_chp_to_load_hrly[s:e], chp_power_to_load[s:e], \
                     where=   pv_plus_chp_to_load_hrly[s:e] >= chp_power_to_load[s:e] , facecolor = 'red', alpha = 0.5 )
    plt.fill_between(hours[s:e], chp_power_to_load[s:e], one[s:e], \
                     where=  chp_power_to_load[s:e] >= one[s:e] , facecolor = 'blue', alpha = 0.5 )
    plt.xlabel('Hours')
    plt.ylabel('Electric Power [kWe]')
#show DG power flow to grid versus load (ignore battery)
    plt.figure() 
    line1, = plt.plot(hours[s:e], chp_elec_to_load_hrly[s:e], 'b',  label = "CHP to load")
    line2, = plt.plot(hours[s:e], chp_elec_to_grid_hrly[s:e], 'b--',  label = "CHP to grid")
    line3, = plt.plot(hours[s:e], pv_elec_to_load_hrly[s:e], 'g', label = "PV to load")
    line4, = plt.plot(hours[s:e], pv_elec_to_grid_hrly[s:e], 'g--', label = "PV to grid")
    plt.legend(handles = [line1, line2, line3, line4])
    plt.xlabel('Hours')
    plt.ylabel('Electric Power [kWe]')
#-----end-----
elif mode == 2:
    grid_elec_to_load_hrly = electric_demand - chp_power
    #show load profile and distribution amongst DG techs
    plt.figure() 
    line1, = plt.plot(hours[s:e], electric_demand[s:e],'k', label = "Demand")
    line2, = plt.plot(hours[s:e], chp_power[s:e], 'b',  label = "CHP")
    #line3, = plt.plot(hours[s:e], grid_elec_to_load_hrly[s:e], 'y', label = "Grid")
    plt.fill_between(hours[s:e], chp_power[s:e], electric_demand[s:e], \
                     where=   electric_demand[s:e] >= chp_power[s:e] , facecolor = 'green', alpha = 0.5 )
    plt.fill_between(hours[s:e], chp_power[s:e], one[s:e], \
                     where=  chp_power[s:e] >= one[s:e] , facecolor = 'blue', alpha = 0.5 )
    plt.legend(handles = [line1, line2])
    plt.xlabel('Hours')
    plt.ylabel('Electric Power [kWe]')
    #grid export?
    plt.figure() 
    line1, = plt.plot(hours[s:e], chp_elec_to_load_hrly[s:e], 'b',  label = "CHP to load")
    line2, = plt.plot(hours[s:e], chp_elec_to_grid_hrly[s:e], 'b--',  label = "CHP to grid")
    plt.legend(handles = [line1, line2])
    plt.xlabel('Hours')
    plt.ylabel('Electric Power [kWe]')
    
    

    
#--------compare electric profile------
base_chp_elec_to_load_hrly = np.array(base_api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_to_load_series_kw']) #CHP elec produciton to load (hourly)
#base_grid_elec_to_load_hrly = bau_grid_load_hrly - base_chp_elec_to_load_hrly
base_pv_elec_to_load_hrly = np.array(base_api_response['outputs']['Scenario']['Site']['PV']
                                   ['year_one_to_load_series_kw'])
base_chp_plus_pv = base_chp_elec_to_load_hrly + base_pv_elec_to_load_hrly

#show load profile and distribution amongst DG techs
plt.figure() 
line1, = plt.plot(hours[s:e], base_chp_elec_to_load_hrly[s:e],'r', label = "Base")
line2, = plt.plot(hours[s:e], chp_elec_to_load_hrly[s:e], 'b',  label = "Corr") #see uf sizes are different
plt.legend(handles = [line1, line2])
plt.xlabel('Hours')
plt.ylabel('Electric Power [kWe]')
#electric efficiency comparison
plt.figure() 
line2, = plt.plot(hours[s:e], base_year_one_elec_eff_series_percent[s:e], 'b',  label = "Base") #see uf sizes are different
line3, = plt.plot(hours[s:e], year_one_elec_eff_series_percent_corr[s:e], 'g',  label = "Corr")
plt.legend(handles = [line2, line3])
plt.xlabel('Hours')
plt.ylabel('Electric Efficiency [%]')
#compare electric profile
plt.figure() 
line1, = plt.plot(hours[s:e], electric_demand[s:e],'k', label = "Demand") 
line2, = plt.plot(hours[s:e], chp_power_to_load[s:e], 'b',  label = "CHP")
line3, = plt.plot(hours[s:e], pv_plus_chp_to_load_hrly[s:e], 'r', label = "PV")
#line4, = plt.plot(hours[s:e], grid_elec_to_load_hrly[s:e], 'r', label = "Grid")
line5, = plt.plot(hours[s:e], base_chp_elec_to_load_hrly[s:e], 'b--', label = 'CHP base')
line6, = plt.plot(hours[s:e], base_chp_plus_pv[s:e], 'r--', label = 'PV base')
fill3 = plt.fill_between(hours[s:e], pv_plus_chp_to_load_hrly[s:e], electric_demand[s:e], \
                    where=   electric_demand[s:e] +0.01 >= pv_plus_chp_to_load_hrly[s:e] , facecolor = 'green', alpha = 0.5 )
fill2 = plt.fill_between(hours[s:e], pv_plus_chp_to_load_hrly[s:e], chp_power_to_load[s:e], \
                     where=   pv_plus_chp_to_load_hrly[s:e] >= chp_power_to_load[s:e] , facecolor = 'red', alpha = 0.5 )  
fill1 = plt.fill_between(hours[s:e], chp_power_to_load[s:e], one[s:e], \
                     where=  chp_power_to_load[s:e] >= one[s:e] , facecolor = 'blue', alpha = 0.5 )
plt.xlabel('Hours')
plt.ylabel('Electric Power [kWe]')
plt.legend(handles = [line2, line5, line3, line6, line1])
#plt.savefig('figures/CHI/CHI_ICE-PV_compare_e_profile.png', dpi = 300)
    
    
    
    
    
 #compare elec dispatch profile with battery
if mode == 3:
    #base
    base_chp_elec_to_load_hrly = np.array(base_api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_to_load_series_kw']) 
    base_pv_elec_to_load_hrly = np.array(base_api_response['outputs']['Scenario']['Site']['PV']
                                   ['year_one_to_load_series_kw'])
    base_bess_elec_to_load_hrly = bess_elec_to_load_hrly = np.array(api_response['outputs']['Scenario']['Site']['Storage']
                                   ['year_one_to_load_series_kw'])
    base_chp_plus_pv = base_chp_elec_to_load_hrly + base_pv_elec_to_load_hrly
    base_chp_plus_pv_plus_bess = base_chp_elec_to_load_hrly + base_pv_elec_to_load_hrly + base_bess_elec_to_load_hrly
    #PV
    pv_elec_total_hrly = np.array(api_response['outputs']['Scenario']['Site']['PV']
                                   ['year_one_power_production_series_kw']) #PV hourly total elec production
    pv_elec_to_load_hrly = np.array(api_response['outputs']['Scenario']['Site']['PV']
                                   ['year_one_to_load_series_kw']) #PV hourly elec production to load
    pv_elec_to_grid_hrly = np.array(api_response['outputs']['Scenario']['Site']['PV']
                                   ['year_one_to_grid_series_kw']) #PV hourly elec production to grid
    #BESS
    bess_SOC_hrly = np.array(api_response['outputs']['Scenario']['Site']['Storage']
                                   ['year_one_soc_series_pct']) #bess state of charge
    bess_elec_to_load_hrly = np.array(api_response['outputs']['Scenario']['Site']['Storage']
                                   ['year_one_to_load_series_kw']) #bess hourly elec production to load
    bess_elec_to_grid_hrly = np.array(api_response['outputs']['Scenario']['Site']['Storage']
                                   ['year_one_to_grid_series_kw']) #bess hourly elec production to grid
    print(bess_elec_to_grid_hrly)
    #Grid
    grid_elec_to_load_hrly = electric_demand - chp_power_to_load - pv_elec_to_load_hrly - bess_elec_to_load_hrly
    pv_plus_chp_to_load_hrly = chp_power_to_load + pv_elec_to_load_hrly
    pv_plus_chp_plus_bess_to_load_hrly = chp_power_to_load + pv_elec_to_load_hrly + bess_elec_to_load_hrly
    
    #ELEC Dispatch
    plt.figure() 
    line1, = plt.plot(hours[s:e], electric_demand[s:e],'k', label = "Demand", linewidth=2)
    line2, = plt.plot(hours[s:e], base_chp_elec_to_load_hrly[s:e], 'b--',  label = "CHP base", linewidth = 1)
    line3, = plt.plot(hours[s:e], base_chp_plus_pv[s:e], 'r--', label = "PV base", linewidth =1)
    line4, = plt.plot(hours[s:e], base_chp_plus_pv_plus_bess[s:e], 'y--', label = "BESS")
    fill3 = plt.fill_between(hours[s:e], pv_plus_chp_to_load_hrly[s:e], pv_plus_chp_plus_bess_to_load_hrly[s:e], \
                     where=   pv_plus_chp_plus_bess_to_load_hrly[s:e] >= pv_plus_chp_to_load_hrly[s:e] , facecolor = 'yellow', alpha = 0.5, label = 'BESS' )
    fill4 = plt.fill_between(hours[s:e], pv_plus_chp_plus_bess_to_load_hrly[s:e], electric_demand[s:e], \
                     where=   electric_demand[s:e] +0.01 >= pv_plus_chp_plus_bess_to_load_hrly[s:e] , facecolor = 'green', alpha = 0.5, label = 'Grid' )
    fill2 = plt.fill_between(hours[s:e], pv_plus_chp_to_load_hrly[s:e], chp_power_to_load[s:e], \
                     where=   pv_plus_chp_to_load_hrly[s:e] >= chp_power_to_load[s:e] , facecolor = 'red', alpha = 0.5, label = 'PV' )
    fill1 = plt.fill_between(hours[s:e], chp_power_to_load[s:e], one[s:e], \
                     where=  chp_power_to_load[s:e] >= one[s:e] , facecolor = 'blue', alpha = 0.5, label = 'CHP' )
    plt.xlabel('Hours')
    plt.ylabel('Electric Power [kWe]')
    plt.legend(handles = [line1, line2, line3, line4])
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

chp_therm_to_load_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                      ['year_one_thermal_to_load_series_mmbtu_per_hour'], index = dti)
chp_therm_to_TES_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                     ['year_one_thermal_to_tes_series_mmbtu_per_hour'], index = dti)
chp_therm_total_hrly = chp_therm_to_load_hrly + chp_therm_to_TES_hrly
chp_fuel_annual = api_response['outputs']['Scenario']['Site']['CHP']['year_one_fuel_used_mmbtu']
chp_elec_effic = np.sum(chp_elec_total_hrly)/(chp_fuel_annual * 1E6/3412.0)
chp_total_effic = (np.sum(chp_elec_total_hrly) + np.sum(chp_therm_total_hrly)* 1E6/3412.0)/(chp_fuel_annual * 1E6/3412.0)
chp_elec_cf = np.sum(chp_elec_total_hrly)/(chp_size_kwe*8760)
chp_avg_load_when_on = chp_elec_total_hrly[chp_elec_total_hrly>1].mean()
boiler_fuel_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['Boiler']
                                ['year_one_boiler_fuel_consumption_series_mmbtu_per_hr'], index = dti)
boiler_fuel_annual = boiler_fuel_hrly.resample('Y').sum()

#-----chris-plot thermal load profiles----
boiler_therm_total_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['Boiler']
                                     ['year_one_boiler_thermal_production_series_mmbtu_per_hr'], index = dti)
boiler_fuel_hrly_bau = pd.DataFrame(api_response['outputs']['Scenario']['Site']['LoadProfileBoilerFuel']
                                     ['year_one_boiler_fuel_load_series_mmbtu_per_hr_bau'], index = dti)
therm_load_hrly = boiler_fuel_hrly_bau*boiler_eff


chp_tp = np.array(api_response['outputs']['Scenario']['Site']['CHP']['year_one_thermal_to_load_series_mmbtu_per_hour'])
base_chp_tp = np.array(base_api_response['outputs']['Scenario']['Site']['CHP']['year_one_thermal_to_load_series_mmbtu_per_hour'])
boiler_tp = np.array(api_response['outputs']['Scenario']['Site']['Boiler']['year_one_boiler_thermal_production_series_mmbtu_per_hr'])
#thermal_demand = np.array(api_response['outputs']['Scenario']['Site']['LoadProfileBoilerFuel']
#                          ['year_one_boiler_fuel_load_series_mmbtu_per_hr_bau'])/boiler_eff
thermal_demand = chp_tp+boiler_tp

plt.figure()
line1, = plt.plot(hours[s:e], chp_tp[s:e], 'b',  label = "CHP")
#line2, = plt.plot(hours[s:e], boiler_tp[s:e], 'r',  label = "Boiler")
#line3, = plt.plot(hours[s:e], HotTES_to_load_hrly[s:e], 'y',  label = "TES")
line4, = plt.plot(hours[s:e], thermal_demand[s:e], 'k',  label = "Demand")
line5, = plt.plot(hours[s:e], base_chp_tp[s:e], 'b--',  label = "CHP base")
plt.legend(handles = [line1, line5, line4])
plt.fill_between(hours[s:e], chp_tp[s:e],thermal_demand[s:e], \
                     where=   thermal_demand[s:e] >= chp_tp[s:e] , facecolor = 'green', alpha = 0.5 )
plt.fill_between(hours[s:e], chp_tp[s:e], one[s:e], \
                     where=  chp_tp[s:e] >= one[s:e] , facecolor = 'blue', alpha = 0.5 )
plt.xlabel('Hours')
plt.ylabel('Thermal Production [MMBtu/hr]')
#plt.savefig('figures/CHI/CHI_ICE-PV_compare_tp_profile.png', dpi = 300)




#------compare thermal profiles-------
base_chp_therm_to_load_hrly = pd.DataFrame(base_api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_thermal_to_load_series_mmbtu_per_hour'], index = dti) #CHP elec produciton to load (hourly)
#show load profile and distribution amongst DG techs
plt.figure() 
line1, = plt.plot(hours[s:e], base_chp_therm_to_load_hrly[s:e],'r', label = "Base")
line2, = plt.plot(hours[s:e], chp_therm_to_load_hrly[s:e], 'b',  label = "Corr") #see uf sizes are different
line3, = plt.plot(hours[s:e], CHP_max_heat_full_load*np.ones(e-s), 'b--',  label = "Rated (Corr)")
plt.legend(handles = [line1, line2, line3])
plt.xlabel('Hours')
plt.ylabel('Thermal Production [MMBtu/hr]')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Times New Roman']


plt.figure() 
line2, = plt.plot(hours[s:e], T_ISO*np.ones(e-s), 'b',  label = "ISO") #see if sizes are different
line3, = plt.plot(hours[s:e], ambient_temperature[s:e], 'g',  label = "TMY3")
plt.legend(handles = [line2, line3])
plt.xlabel('Hours')
plt.ylabel('Ambient Temperature [F]')


#chp vs boiler fuel consumption
base_year_one_chp_fuel = base_api_response['outputs']['Scenario']['Site']['CHP']['year_one_fuel_used_mmbtu']
year_one_chp_fuel = api_response['outputs']['Scenario']['Site']['CHP']['year_one_fuel_used_mmbtu']
print("Annual CHP Fuel Use (Base) =", '%.3f' %base_year_one_chp_fuel)
print("Annual CHP Fuel Use (Corr) =", '%.3f' %year_one_chp_fuel)
base_year_one_boiler_fuel = base_api_response['outputs']['Scenario']['Site']['Boiler']['year_one_boiler_fuel_consumption_mmbtu']
year_one_boiler_fuel = api_response['outputs']['Scenario']['Site']['Boiler']['year_one_boiler_fuel_consumption_mmbtu']
print("Annual Boiler Fuel Use (Base) =", '%.3f' %base_year_one_boiler_fuel)
print("Annual Boiler Fuel Use (Corr) =", '%.3f' %year_one_boiler_fuel)



In [ ]:
# Look at Staging Server runs
run_id = "85023f4f-a149-4df8-80cc-35301600691c" #"28e6404d-b977-47b9-95c8-43eb74a0f2fc"
API_KEY = 'uaz52dr5KTT5Qs5U72rS70hw3IYeHVEyAaDUFQvo'
root_url = 'https://chpbeta1-reopt-stage-api.its.nrel.gov'
results_url = root_url + '/v1/job/<run_uuid>/results/?api_key=' + API_KEY
url = results_url.replace('<run_uuid>', run_id)
#url = "https://chp-reopt-stage.its.nrel.gov/tool/results/28e6404d-b977-47b9-95c8-43eb74a0f2fc"
response = requests.get(url=url)
resp_dict = json.loads(response.text)

SAVE data to pickle file

In [ ]:
save_directory = 'saved_results_6/'
pickle_file = 'NYC_MT-PV_base-corr_hospital_gap1'
with open(save_directory + pickle_file + '.pickle', 'wb') as handle:
    data = {'API_response': api_response, 'scenario_data': post_1}
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Data process scenario with BESS

In [ ]:
from scipy.stats import linregress
chp_elec_eff_full_load = api_response['inputs']['Scenario']['Site']['CHP']['elec_effic_full_load']
chp_elec_eff_half_load = api_response['inputs']['Scenario']['Site']['CHP']['elec_effic_half_load']
chp_therm_eff_full_load = api_response['inputs']['Scenario']['Site']['CHP']['thermal_effic_full_load']
chp_therm_eff_half_load = api_response['inputs']['Scenario']['Site']['CHP']['thermal_effic_half_load']
chp_size_kwe = api_response['outputs']['Scenario']['Site']['CHP']['size_kw']
CHP_fuel_burn_full_load = chp_size_kwe / chp_elec_eff_full_load * 3412/1e6 #[MMBtu/hr]
CHP_fuel_burn_half_load = 0.5*chp_size_kwe / chp_elec_eff_half_load * 3412/1e6
slope, intercept, rvalue, pvalue, stderr = linregress([0.5*chp_size_kwe, chp_size_kwe], [CHP_fuel_burn_half_load, CHP_fuel_burn_full_load])
CHP_elec_slope = slope
CHP_elec_intercept = intercept
CHP_max_heat_full_load = CHP_fuel_burn_full_load * chp_therm_eff_full_load #[MMBtu/hr]
CHP_max_heat_half_load = CHP_fuel_burn_half_load * chp_therm_eff_half_load 
slope_tp, intercept_tp, rvalue, pvalue, stderr = linregress([0.5*chp_size_kwe, chp_size_kwe], [CHP_max_heat_half_load, CHP_max_heat_full_load])
CHP_therm_slope = slope_tp
CHP_therm_intercept = intercept_tp
print(CHP_elec_slope)
print(CHP_elec_intercept)
print(CHP_therm_slope)
print(CHP_therm_intercept)


load_year = str(api_response['inputs']['Scenario']['Site']['LoadProfile']['year'])
dti = pd.date_range( load_year +'-01-01', periods=8760, freq='H')
bau_grid_load_less_chiller_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']
                                               ['LoadProfile']['year_one_electric_load_series_kw'], index = dti)
bau_boiler_load_fuel_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']
                                         ['LoadProfileBoilerFuel']['year_one_boiler_fuel_load_series_mmbtu_per_hr_bau'], index = dti)
bau_chiller_load_elec_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']
                                          ['LoadProfileChillerElectric']['year_one_chiller_electric_load_series_kw'], index = dti)
boiler_eff = api_response['inputs']['Scenario']['Site']['Boiler']['boiler_efficiency']

bau_grid_load_hrly = bau_grid_load_less_chiller_hrly + bau_chiller_load_elec_hrly
bau_grid_load_peak = np.max(bau_grid_load_hrly)
bau_grid_load_mean = np.mean(bau_grid_load_hrly)
bau_grid_load_total = np.sum(bau_grid_load_hrly)
bau_therm_load_peak = np.max(bau_boiler_load_fuel_hrly)*boiler_eff #converted units of fuel to units of heat
bau_therm_load_mean = np.mean(bau_boiler_load_fuel_hrly)*boiler_eff
bau_therm_load_total = np.sum(bau_boiler_load_fuel_hrly)*boiler_eff
chp_therm_to_elec_full_load = chp_therm_eff_full_load/chp_elec_eff_full_load


chp_elec_total_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_electric_production_series_kw'], index = dti) #total CHP elec production (hourly)
chp_elec_to_load_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_to_load_series_kw'], index = dti) #CHP elec produciton to load (hourly)
chp_elec_to_grid_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_to_grid_series_kw'], index = dti) #CHP elec production to grid (hourly)

grid_load_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']
                                               ['LoadProfile']['year_one_electric_load_series_kw'], index = dti)







#-----chris-calc electric efficiency----
slope_ee, intercept_ee, rvalue, pvalue, stderr = linregress([0.5*chp_size_kwe, chp_size_kwe], [100*chp_elec_eff_half_load, 100*chp_elec_eff_full_load])
print("Electric Efficiency Slope = ", '%.4f' %slope_ee)
print("Electric Efficiency Intercept = ", '%.1f' %intercept_ee)        
year_one_elec_eff_series_percent = slope_ee*np.array(chp_elec_total_hrly.values)+ intercept_ee

# fuel burn
slope_fb, intercept_fb, rvalue, pvalue, stderr = linregress([0.5*chp_size_kwe, chp_size_kwe], [CHP_fuel_burn_half_load, CHP_fuel_burn_full_load])
corr = 1 # 1 = MT corr , 2 = ICE corr, else = base
cf_fb = np.ones(8760)
t_ref = 77  # [deg F]
if corr == 1:
    T_ISO = 59
    for i in range(8760):
        cf_fb[i] = 0.9529*np.exp(0.0008249*ambient_temperature[i]) - (7.496*(10**(-6)))*np.exp(0.06726*ambient_temperature[i])
if corr == 2:
    T_ISO = 77
    for i in range(8760):
        if ambient_temperature[i] <= t_ref:
                 cf_fb[i] = 1 + 0.0003*(ambient_temperature[i] - t_ref)
        else:
                 cf_fb[i] = 1
else:
    T_ISO = 59
    
chp_elec_total_hrly_a = np.array(chp_elec_total_hrly)
year_one_fuel_burn_series = np.ones(8760)
year_one_elec_eff_series_percent_corr = np.ones(8760)
for i in range(8760):
	year_one_fuel_burn_series[i] = cf_fb[i]*(slope_fb*chp_elec_total_hrly_a[i] + intercept_fb)
for i in range(8760):    
    year_one_elec_eff_series_percent_corr[i] = 100*(chp_elec_total_hrly_a[i] / (year_one_fuel_burn_series[i]* (1e6/3412)))    


#thermal production
year_one_thermal_prod_series_corr = np.ones(8760)
cf_tp = np.ones(8760)
if corr == 1:
    for i in range(8760):
        cf_tp[i] = 0.005852*ambient_temperature[i] + 0.6615
if corr == 2:
    for i in range(8760):
        if ambient_temperature[i] <= t_ref:
                 cf_tp[i] = 1 + 0.001*(ambient_temperature[i] - t_ref)
        else:
                 cf_tp[i] = 1
for i in range(8760):
	year_one_thermal_prod_series_corr[i] = cf_tp[i]*(slope_tp*chp_elec_total_hrly_a[i] + intercept_tp)                
                                                                                               

        
        
        
        
        
        
        
        
        
        
        
hours = np.linspace(1,8760,8760)
s = 8000 #start hour mark
e = 8100 #end hour mark
plt.figure()
plt.plot(hours[s:e], year_one_elec_eff_series_percent[s:e], color='green') #linestyle='none',marker='o') 
#------end----

electric_demand = np.array(api_response['outputs']['Scenario']['Site']['LoadProfile']['year_one_electric_load_series_kw'])
chp_power = np.array(api_response['outputs']['Scenario']['Site']['CHP']['year_one_electric_production_series_kw'])
chp_power_to_load = np.array(api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_to_load_series_kw'])
chp_power_to_grid = np.array(api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_to_grid_series_kw'])
one = np.zeros(8760)

save = 2 # 1 = save , 2 = do not save
mode = 3 #CHP+PV = 1 , CHP = 2, CHP+PV+BES = 3
if mode == 1:
#-----chris-plot electric load profiles----
    pv_elec_total_hrly = np.array(api_response['outputs']['Scenario']['Site']['PV']
                                   ['year_one_power_production_series_kw']) #PV hourly total elec production
    pv_elec_to_load_hrly = np.array(api_response['outputs']['Scenario']['Site']['PV']
                                   ['year_one_to_load_series_kw']) #PV hourly elec production to load
    pv_elec_to_grid_hrly = np.array(api_response['outputs']['Scenario']['Site']['PV']
                                   ['year_one_to_grid_series_kw']) #PV hourly elec production to grid
    grid_elec_to_load_hrly = electric_demand - chp_power_to_load - pv_elec_to_load_hrly
    pv_plus_chp_to_load_hrly = chp_power_to_load + pv_elec_to_load_hrly

#show load profile and distribution amongst DG techs
    plt.figure() 
    line1, = plt.plot(hours[s:e], electric_demand[s:e],'k', label = "Demand")
    line2, = plt.plot(hours[s:e], chp_power_to_load[s:e], 'b',  label = "CHP")
    line3, = plt.plot(hours[s:e], pv_plus_chp_to_load_hrly[s:e], 'r', label = "PV")
    #line4, = plt.plot(hours[s:e], grid_elec_to_load_hrly[s:e], 'r', label = "Grid")
    fill3 = plt.fill_between(hours[s:e], pv_plus_chp_to_load_hrly[s:e], electric_demand[s:e], \
                     where=   electric_demand[s:e] >= pv_plus_chp_to_load_hrly[s:e] , facecolor = 'green', alpha = 0.5, label = 'Grid' )
    fill2 = plt.fill_between(hours[s:e], pv_plus_chp_to_load_hrly[s:e], chp_power_to_load[s:e], \
                     where=   pv_plus_chp_to_load_hrly[s:e] >= chp_power_to_load[s:e] , facecolor = 'red', alpha = 0.5, label = 'PV' )
    fill1 = plt.fill_between(hours[s:e], chp_power_to_load[s:e], one[s:e], \
                     where=  chp_power_to_load[s:e] >= one[s:e] , facecolor = 'blue', alpha = 0.5, label = 'CHP' )
    plt.xlabel('Hours')
    plt.ylabel('Electric Power [kWe]')
    plt.legend(handles = [fill1, fill2, fill3, line1])
    if save == 1:
        plt.savefig('figures/CHI/CHI_ICE-PV_corr_e_profile_summer.png', dpi = 300)
#show DG power flow to grid versus load (ignore battery)
    plt.figure() 
    line1, = plt.plot(hours[s:e], chp_elec_to_load_hrly[s:e], 'b',  label = "CHP to load")
    line2, = plt.plot(hours[s:e], chp_elec_to_grid_hrly[s:e], 'b--',  label = "CHP to grid")
    line3, = plt.plot(hours[s:e], pv_elec_to_load_hrly[s:e], 'g', label = "PV to load")
    line4, = plt.plot(hours[s:e], pv_elec_to_grid_hrly[s:e], 'g--', label = "PV to grid")
    plt.legend(handles = [line1, line2, line3, line4])
    plt.xlabel('Hours')
    plt.ylabel('Electric Power [kWe]')
    
elif mode == 2:
    grid_elec_to_load_hrly = electric_demand - chp_power
    #show load profile and distribution amongst DG techs
    plt.figure() 
    line1, = plt.plot(hours[s:e], electric_demand[s:e],'k', label = "Demand")
    line2, = plt.plot(hours[s:e], chp_power[s:e], 'b',  label = "CHP")
    #line3, = plt.plot(hours[s:e], grid_elec_to_load_hrly[s:e], 'y', label = "Grid")
    fill2 = plt.fill_between(hours[s:e], chp_power[s:e], electric_demand[s:e], \
                     where=   electric_demand[s:e] >= chp_power[s:e] , facecolor = 'green', alpha = 0.5, label = 'Grid' )
    fill1 = plt.fill_between(hours[s:e], chp_power[s:e], one[s:e], \
                     where=  chp_power[s:e] >= one[s:e] , facecolor = 'blue', alpha = 0.5, label = 'CHP' )
    plt.legend(handles = [fill1, fill2, line2])
    plt.xlabel('Hours')
    plt.ylabel('Electric Power [kWe]')
    #grid export?
    plt.figure() 
    line1, = plt.plot(hours[s:e], chp_elec_to_load_hrly[s:e], 'b',  label = "CHP to load")
    line2, = plt.plot(hours[s:e], chp_elec_to_grid_hrly[s:e], 'b--',  label = "CHP to grid")
    plt.legend(handles = [line1, line2])
    plt.xlabel('Hours')
    plt.ylabel('Electric Power [kWe]')

elif mode == 3:
    #PV
    pv_elec_total_hrly = np.array(api_response['outputs']['Scenario']['Site']['PV']
                                   ['year_one_power_production_series_kw']) #PV hourly total elec production
    pv_elec_to_load_hrly = np.array(api_response['outputs']['Scenario']['Site']['PV']
                                   ['year_one_to_load_series_kw']) #PV hourly elec production to load
    pv_elec_to_grid_hrly = np.array(api_response['outputs']['Scenario']['Site']['PV']
                                   ['year_one_to_grid_series_kw']) #PV hourly elec production to grid
    #BESS
    bess_SOC_hrly = np.array(api_response['outputs']['Scenario']['Site']['Storage']
                                   ['year_one_soc_series_pct']) #bess state of charge
    bess_elec_to_load_hrly = np.array(api_response['outputs']['Scenario']['Site']['Storage']
                                   ['year_one_to_load_series_kw']) #bess hourly elec production to load
    bess_elec_to_grid_hrly = np.array(api_response['outputs']['Scenario']['Site']['Storage']
                                   ['year_one_to_grid_series_kw']) #bess hourly elec production to grid
    print(bess_elec_to_grid_hrly)
    #Grid
    grid_elec_to_load_hrly = electric_demand - chp_power_to_load - pv_elec_to_load_hrly - bess_elec_to_load_hrly
    pv_plus_chp_to_load_hrly = chp_power_to_load + pv_elec_to_load_hrly
    pv_plus_chp_plus_bess_to_load_hrly = chp_power_to_load + pv_elec_to_load_hrly + bess_elec_to_load_hrly
    
    #ELEC Dispatch
    plt.figure() 
    line1, = plt.plot(hours[s:e], electric_demand[s:e],'k', label = "Demand", linewidth=2)
    #line2, = plt.plot(hours[s:e], chp_power_to_load[s:e], 'b',  label = "CHP", linewidth = 1)
    #line3, = plt.plot(hours[s:e], pv_plus_chp_to_load_hrly[s:e], 'r', label = "PV", linewidth =1)
    #line4, = plt.plot(hours[s:e], pv_plus_chp_plus_bess_to_load_hrly[s:e], 'y', label = "BESS")
    fill3 = plt.fill_between(hours[s:e], pv_plus_chp_to_load_hrly[s:e], pv_plus_chp_plus_bess_to_load_hrly[s:e], \
                     where=   pv_plus_chp_plus_bess_to_load_hrly[s:e] >= pv_plus_chp_to_load_hrly[s:e] , facecolor = 'yellow', alpha = 0.5, label = 'BESS' )
    fill4 = plt.fill_between(hours[s:e], pv_plus_chp_plus_bess_to_load_hrly[s:e], electric_demand[s:e], \
                     where=   electric_demand[s:e] +0.01 >= pv_plus_chp_plus_bess_to_load_hrly[s:e] , facecolor = 'green', alpha = 0.5, label = 'Grid' )
    fill2 = plt.fill_between(hours[s:e], pv_plus_chp_to_load_hrly[s:e], chp_power_to_load[s:e], \
                     where=   pv_plus_chp_to_load_hrly[s:e] >= chp_power_to_load[s:e] , facecolor = 'red', alpha = 0.5, label = 'PV' )
    fill1 = plt.fill_between(hours[s:e], chp_power_to_load[s:e], one[s:e], \
                     where=  chp_power_to_load[s:e] >= one[s:e] , facecolor = 'blue', alpha = 0.5, label = 'CHP' )
    plt.xlabel('Hours')
    plt.ylabel('Electric Power [kWe]')
    plt.legend(handles = [fill1, fill2, fill3, fill4, line1])
    #SAVE
    #plt.savefig('figures_5/SF/SF_ICE-PV-BESS-ABS_base2_ELECprofile_summer.png', dpi = 300)
    if save == 1:
        plt.savefig('figures/CHI/CHI_ICE-PV_corr_e_profile_summer.png', dpi = 300)  

        
    #calculate CHP distributions
    chp_power_to_battery = np.array(api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_to_battery_series_kw']) #PV hourly elec production to grid
    chp_power_to_load_plus_grid = chp_power_to_load + chp_power_to_grid
    plt.figure() 
    #line1, = plt.plot(hours[s:e], chp_power_to_load[s:e], 'b',  label = "CHP to load")
    #line2, = plt.plot(hours[s:e], chp_power_to_load_plus_grid[s:e], 'g',  label = "CHP to grid")
    #line3, = plt.plot(hours[s:e], chp_elec_to_bess_hrly[s:e], 'b',  label = "CHP to BESS")
    line4, = plt.plot(hours[s:e], chp_power[s:e], 'k',  label = "Total CHP", linewidth=2)
    
    fill1 = plt.fill_between(hours[s:e], chp_power_to_load[s:e], one[s:e], \
                     where=  chp_power_to_load[s:e] >= one[s:e] , facecolor = 'blue', alpha = 0.5, label = 'CHP to Load' )
    fill2 = plt.fill_between(hours[s:e], chp_power_to_load[s:e], chp_power_to_load_plus_grid[s:e], \
                     where=  chp_power_to_load_plus_grid[s:e] >= chp_power_to_load[s:e] , facecolor = 'green', alpha = 0.5, label = 'CHP to Grid' )
    fill3 = plt.fill_between(hours[s:e], chp_power_to_load_plus_grid[s:e], chp_power[s:e], \
                     where=  chp_power[s:e] >= chp_power_to_load_plus_grid[s:e] , facecolor = 'yellow', alpha = 0.75, label = 'CHP to BESS' )
    plt.xlabel('Hours')
    plt.ylabel('Electric Power [kWe]')
    plt.legend(handles = [fill1, fill2, fill3, line4])
    #SAVE
    #plt.savefig('figures_5/SF/SF_ICE-PV-BESS_base_CHPprofile_summer.png', dpi = 300)
    
    
    #show PV power distribution
    pv_elec_to_battery_hrly = np.array(api_response['outputs']['Scenario']['Site']['PV']
                                   ['year_one_to_battery_series_kw']) #PV hourly elec production to grid
    pv_elec_to_load_plus_grid_hrly = pv_elec_to_load_hrly + pv_elec_to_grid_hrly
    plt.figure()
    #line1, = plt.plot(hours[s:e], pv_elec_to_load_hrly[s:e], 'b', label = "PV to load")
    #line2, = plt.plot(hours[s:e], pv_elec_to_load_plus_grid_hrly[s:e], 'g', label = "PV to Grid")
    #line3, = plt.plot(hours[s:e], pv_elec_to_battery_hrly[s:e], 'g--', label = "PV to BESS")
    line4, = plt.plot(hours[s:e], pv_elec_total_hrly[s:e], 'k',  label = "Total PV", linewidth=2)
    fill1 = plt.fill_between(hours[s:e], pv_elec_to_load_hrly[s:e], one[s:e], \
                     where=  pv_elec_to_load_hrly[s:e] >= one[s:e] , facecolor = 'blue', alpha = 0.5, label = 'PV to Load' )
    fill2 = plt.fill_between(hours[s:e], pv_elec_to_load_hrly[s:e], pv_elec_to_load_plus_grid_hrly[s:e], \
                     where=  pv_elec_to_load_plus_grid_hrly[s:e] >= pv_elec_to_load_hrly[s:e] , facecolor = 'green', alpha = 0.5, label = 'PV to Grid' )
    fill3 = plt.fill_between(hours[s:e], pv_elec_to_load_plus_grid_hrly[s:e], pv_elec_total_hrly[s:e], \
                     where=  pv_elec_total_hrly[s:e] >= pv_elec_to_load_plus_grid_hrly[s:e] , facecolor = 'yellow', alpha = 0.75, label = 'PV to BESS' )

    plt.legend(handles = [fill1, fill2, fill3, line4])
    plt.xlabel('Hours')
    plt.ylabel('Electric Power [kWe]')
    #SAVE
    #plt.savefig('figures_5/SF/SF_ICE-PV-BESS_base_PVprofile_summer.png', dpi = 300)
    
    
    ##show BESS power distribution
    #line3, = plt.plot(hours[s:e], bess_elec_to_load_hrly[s:e], 'r', label = "BESS to load")
    ##line6, = plt.plot(hours[s:e], bess_elec_to_grid_hrly[s:e], 'r--', label = "BESS to grid")
    #plt.legend(handles = [line1, line2, line3, line4, line5])
    #plt.xlabel('Hours')
    #plt.ylabel('Electric Power [kWe]')
    
    
    #BESS state of charge
    plt.figure()
    line1, =  plt.plot(hours[s:e], 100*bess_SOC_hrly[s:e], 'r', label = "BESS")
    plt.legend(handles = [line1])
    plt.xlabel('Hours')
    plt.ylabel('Battery State of Charge [%]')
    
    
    

    
    
    
    
    
    
    
    
    
    #THERMAL DISPATCH
chp_therm_to_load_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                      ['year_one_thermal_to_load_series_mmbtu_per_hour'], index = dti)
chp_therm_to_TES_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                     ['year_one_thermal_to_tes_series_mmbtu_per_hour'], index = dti)
chp_therm_total_hrly = chp_therm_to_load_hrly + chp_therm_to_TES_hrly
chp_fuel_annual = api_response['outputs']['Scenario']['Site']['CHP']['year_one_fuel_used_mmbtu']
chp_elec_effic = np.sum(chp_elec_total_hrly)/(chp_fuel_annual * 1E6/3412.0)
chp_total_effic = (np.sum(chp_elec_total_hrly) + np.sum(chp_therm_total_hrly)* 1E6/3412.0)/(chp_fuel_annual * 1E6/3412.0)
chp_elec_cf = np.sum(chp_elec_total_hrly)/(chp_size_kwe*8760)
chp_avg_load_when_on = chp_elec_total_hrly[chp_elec_total_hrly>1].mean()
boiler_fuel_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['Boiler']
                                ['year_one_boiler_fuel_consumption_series_mmbtu_per_hr'], index = dti)
boiler_fuel_annual = boiler_fuel_hrly.resample('Y').sum()



#-----chris-plot thermal load profiles----
boiler_therm_total_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['Boiler']
                                     ['year_one_boiler_thermal_production_series_mmbtu_per_hr'], index = dti)
boiler_fuel_hrly_bau = pd.DataFrame(api_response['outputs']['Scenario']['Site']['LoadProfileBoilerFuel']
                                     ['year_one_boiler_fuel_load_series_mmbtu_per_hr_bau'], index = dti)
therm_load_hrly = boiler_fuel_hrly_bau*boiler_eff


mode_therm = 1 # 1 = no HOT TES, 2 = HOT TES
if mode_therm == 1:
    chp_tp = np.array(api_response['outputs']['Scenario']['Site']['CHP']['year_one_thermal_to_load_series_mmbtu_per_hour'])
    boiler_tp = np.array(api_response['outputs']['Scenario']['Site']['Boiler']['year_one_boiler_thermal_production_series_mmbtu_per_hr'])
    #thermal_demand = np.array(api_response['outputs']['Scenario']['Site']['LoadProfileBoilerFuel']
    #                          ['year_one_boiler_fuel_load_series_mmbtu_per_hr_bau'])/boiler_eff
    thermal_demand = chp_tp+boiler_tp

    plt.figure()
    line1, = plt.plot(hours[s:e], chp_tp[s:e], 'b',  label = "CHP")
    #line2, = plt.plot(hours[s:e], boiler_tp[s:e], 'r',  label = "Boiler")
    #line3, = plt.plot(hours[s:e], HotTES_to_load_hrly[s:e], 'y',  label = "TES")
    line4, = plt.plot(hours[s:e], thermal_demand[s:e], 'k',  label = "Demand") #really"Total"

    fill2 = plt.fill_between(hours[s:e], chp_tp[s:e],thermal_demand[s:e], \
                     where=   thermal_demand[s:e] >= chp_tp[s:e] , facecolor = 'green', alpha = 0.5, label = 'Boiler' )
    fill1 = plt.fill_between(hours[s:e], chp_tp[s:e], one[s:e], \
                     where=  chp_tp[s:e] >= one[s:e] , facecolor = 'blue', alpha = 0.5, label = 'CHP' )
    plt.xlabel('Hours')
    plt.ylabel('Thermal Production [MMBtu/hr]')
    plt.legend(handles = [fill1, fill2, line4])
    #SAVE
    #plt.savefig('figures_5/SF/SF_ICE-PV-BESS-ABS_base2_HEATprofile_summer.png', dpi = 300)
    if save == 1:
        plt.savefig('figures/CHI/CHI_ICE-PV_corr_tp_profile_summer.png', dpi = 300)
else:
    chp_tp = np.array(api_response['outputs']['Scenario']['Site']['CHP']['year_one_thermal_to_load_series_mmbtu_per_hour'])
    chp_to_HotTES = np.array(api_response['outputs']['Scenario']['Site']['CHP']['year_one_thermal_to_tes_series_mmbtu_per_hour'])
    boiler_tp = np.array(api_response['outputs']['Scenario']['Site']['Boiler']['year_one_boiler_thermal_production_series_mmbtu_per_hr'])
    HotTES_to_load_hrly = np.array(api_response['outputs']['Scenario']['Site']['HotTES']
                                     ['year_one_thermal_from_hot_tes_series_mmbtu_per_hr'])
    thermal_demand = chp_tp + boiler_tp + HotTES_to_load_hrly
    chp_plus_tes_tp = chp_tp + HotTES_to_load_hrly
    
    #THERMAL LOAD DISTRIBUTION
    plt.figure()
    #line1, = plt.plot(hours[s:e], chp_tp[s:e], 'b',  label = "CHP")
    #line2, = plt.plot(hours[s:e], boiler_tp[s:e], 'r',  label = "Boiler")
    #line3, = plt.plot(hours[s:e], HotTES_to_load_hrly[s:e], 'y',  label = "TES")
    line4, = plt.plot(hours[s:e], thermal_demand[s:e], 'k',  label = "Demand", linewidth = 2) #really"Total"
    fill1 = plt.fill_between(hours[s:e], chp_tp[s:e], one[s:e], \
                     where=  chp_tp[s:e] >= one[s:e] , facecolor = 'blue', alpha = 0.5, label = 'CHP' )
    fill2 = plt.fill_between(hours[s:e], chp_plus_tes_tp[s:e], chp_tp[s:e], \
                     where=   chp_plus_tes_tp[s:e] >= chp_tp[s:e] , facecolor = 'yellow', alpha = 0.5, label = 'TES' )
    fill3 = plt.fill_between(hours[s:e], chp_plus_tes_tp[s:e], thermal_demand[s:e], \
                     where=   thermal_demand[s:e] >= chp_plus_tes_tp[s:e] , facecolor = 'green', alpha = 0.5, label = 'Boiler' )
    plt.xlabel('Hours')
    plt.ylabel('Thermal Production [MMBtu/hr]')
    plt.legend(handles = [fill1, fill2, fill3, line4])
    #SAVE
    #plt.savefig('figures_5/SF/SF_ICE-PV-BESS-HOTTES_base_HEATprofile_summer.png', dpi = 300)
    
    #CHP THERMAL DISTRIBUTION
    plt.figure()
    line4, = plt.plot(hours[s:e], chp_tp[s:e]+chp_to_HotTES[s:e], 'k',  label = "Total CHP", linewidth = 2) #really"Total"
    fill1 = plt.fill_between(hours[s:e], chp_tp[s:e], one[s:e], \
                     where=  chp_tp[s:e] >= one[s:e] , facecolor = 'blue', alpha = 0.5, label = 'CHP to Load' )
    fill2 = plt.fill_between(hours[s:e], chp_tp[s:e]+chp_to_HotTES[s:e], chp_tp[s:e], \
                     where=   chp_tp[s:e]+chp_to_HotTES[s:e] >= chp_tp[s:e] , facecolor = 'yellow', alpha = 0.5, label = 'CHP to TES' )
    plt.xlabel('Hours')
    plt.ylabel('Thermal Production [MMBtu/hr]')
    plt.legend(handles = [fill1, fill2, line4])
    #SAVE
    #plt.savefig('figures_5/SF/SF_ICE-PV-BESS-HOTTES_base_CHPHEATprofile_summer.png', dpi = 300)
    
    #HOT TES SOC
    HotTES_SOC_pct = np.array(api_response['outputs']['Scenario']['Site']['HotTES']
                                     ['year_one_hot_tes_soc_series_pct'])
    plt.figure()
    line1, = plt.plot(hours[s:e], 100*HotTES_SOC_pct[s:e], 'k',  label = "Demand", linewidth = 2)
    plt.xlabel('Hours')
    plt.ylabel('Hot TES State of Charge [%]')
    plt.legend(handles = [line1])
    
    
    
    
    
    
    
#check CF calcs
#no TES
plt.figure() 
line2, = plt.plot(hours[s:e], chp_therm_to_load_hrly[s:e], 'g',  label = "Dispatched") #see if sizes are different
line3, = plt.plot(hours[s:e], CHP_max_heat_full_load*np.ones(e-s), 'b',  label = "Rated")
line4, = plt.plot(hours[s:e], year_one_thermal_prod_series_corr[s:e], 'r',  label = "Produced")
plt.legend(handles = [line3, line2, line4])
plt.xlabel('Hours')
plt.ylabel('Thermal Production [MMBtu/hr]')
#plt.ylim([0, 1])
#plt.savefig('figures/CHI/CHI_ICE-PV_corr_tp_profile.png', dpi = 300)

plt.figure() 
line2, = plt.plot(hours[s:e], T_ISO*np.ones(e-s), 'b',  label = "ISO") #see if sizes are different
line3, = plt.plot(hours[s:e], ambient_temperature[s:e], 'g',  label = "TMY3")
plt.legend(handles = [line2, line3])
plt.xlabel('Hours')
plt.ylabel('Ambient Temperature [F]')
#plt.savefig('SF_MT_Tamb.png', dpi = 300)
plt.figure() 
line2, = plt.plot(hours[s:e], chp_size_kwe*np.ones(e-s), 'b',  label = "Rated") #see if sizes are different
line3, = plt.plot(hours[s:e], chp_elec_to_load_hrly[s:e], 'g',  label = "Dispatched")
plt.legend(handles = [line2, line3])
plt.xlabel('Hours')
plt.ylabel('Electric Power [kWe]')
#plt.savefig('figures/PHX/PHX_ICE-PV_corr_e_profile.png', dpi = 300)
plt.figure() 
line2, = plt.plot(hours[s:e], chp_elec_eff_full_load*100*np.ones(e-s), 'b',  label = "Rated") #see if sizes are different
line3, = plt.plot(hours[s:e], year_one_elec_eff_series_percent_corr[s:e], 'g',  label = "Dispatched")
plt.legend(handles = [line2, line3])
plt.xlabel('Hours')
plt.ylabel('Electric Efficiency [%]')
#plt.ylim([36,38])
#plt.savefig('figures/CHI/CHI_ICE-PV_corr_ee_profile.png', dpi = 300)

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Times New Roman']








mode_abs_cooling = 1 # 1 = no abs chiller , 2 = abs chiller
if mode_abs_cooling == 2:
    cooling_load_elec_hrly_bau = np.array(api_response['outputs']['Scenario']['Site']['LoadProfileChillerElectric']['year_one_chiller_electric_load_series_kw_bau'])
    cooling_load_elec_hrly = np.array(api_response['outputs']['Scenario']['Site']['LoadProfileChillerElectric']['year_one_chiller_electric_load_series_kw'])
    cooling_load_therm_hrly = np.array(api_response['outputs']['Scenario']['Site']['LoadProfileChillerElectric']['year_one_chiller_thermal_load_series_ton'])
    ChillerElec_thermal_to_load_hrly = np.array(api_response['outputs']['Scenario']['Site']['ElectricChiller']['year_one_electric_chiller_thermal_to_load_series_ton'])
    ChillerAbs_thermal_to_load_hrly = np.array(api_response['outputs']['Scenario']['Site']['AbsorptionChiller']['year_one_absorp_chl_thermal_to_load_series_ton'])
    ChillerAbs_thermal_consumption_hrly = np.array(api_response['outputs']['Scenario']['Site']['AbsorptionChiller']['year_one_absorp_chl_thermal_consumption_series_mmbtu_per_hr'])
    ChillerElec_plus_ChillerAbs_thermal_to_load_hrly = ChillerElec_thermal_to_load_hrly + ChillerAbs_thermal_to_load_hrly
    print(ChillerElec_thermal_to_load_hrly)
    print(ChillerElec_thermal_to_load_hrly)
    print(ChillerAbs_thermal_to_load_hrly)
    
    plt.figure()
    #line1, = plt.plot(hours[s:e], cooling_load_elec_hrly_bau[s:e]/3.5, 'k', label='Demand', linewidth = 2)
    #line2, = plt.plot(hours[s:e], ChillerElec_thermal_to_load_hrly[s:e], label = 'Elec Chiller')
    #line3, = plt.plot(hours[s:e], ChillerAbs_thermal_to_load_hrly[s:e], label = 'Abs Chiller')
    line4, = plt.plot(hours[s:e], ChillerElec_plus_ChillerAbs_thermal_to_load_hrly[s:e], 'k', label='Demand', linewidth = 2)
    #fill1, = plt.fill_between(hours[s:e], ChillerElec_thermal_to_load_hrly[s:e], one[s:e], where= ChillerElec_thermal_to_load_hrly[s:e] >= one[s:e], facecolor = 'green', alpha = 0.5, label = 'Elec. Chiller to Load')
    #fill2, = plt.fill_between(hours[s:e], ChillerElec_plus_ChillerAbs_thermal_to_load_hrly[s:e], ChillerElec_thermal_to_load_hrly[s:e], where=   ChillerElec_plus_ChillerAbs_thermal_to_load_hrly[s:e] >= ChillerElec_thermal_to_load_hrly[s:e] , facecolor = 'blue', alpha = 0.5, label = 'Abs. Chiller to TES' )
    #fill2, = plt.fill_between(hours[s:e], ChillerElec_plus_ChillerAbs_thermal_to_load_hrly[s:e], ChillerElec_thermal_to_load_hrly[s:e], \
    #                 where=   ChillerElec_plus_ChillerAbs_thermal_to_load_hrly[s:e] >= ChillerElec_thermal_to_load_hrly[s:e] , facecolor = 'blue', alpha = 0.5, label = 'Abs. Chiller to TES' )
    fill2 = plt.fill_between(hours[s:e], ChillerElec_plus_ChillerAbs_thermal_to_load_hrly[s:e], ChillerElec_thermal_to_load_hrly[s:e], \
                     where=   ChillerElec_plus_ChillerAbs_thermal_to_load_hrly[s:e] >= ChillerElec_thermal_to_load_hrly[s:e], facecolor = 'blue', alpha = 0.5, label = 'Abs. Chiller' )

    fill3 = plt.fill_between(hours[s:e], ChillerElec_thermal_to_load_hrly[s:e], one[s:e], \
                     where=   ChillerElec_thermal_to_load_hrly[s:e] >= one[s:e], facecolor = 'green', alpha = 0.5, label = 'Elec. Chiller' )
    plt.xlabel('Hours')
    plt.ylabel('Cooling Production [Tons]')
    plt.legend(handles = [fill2, fill3, line4])
    #SAVE
    #plt.savefig('figures_5/SF/SF_ICE-PV-BESS-ABS_base2_COOLprofile_summer.png', dpi = 300)
    
    s=3000
    e=4000
    plt.figure()
    line3, = plt.plot(hours[s:e], ChillerAbs_thermal_to_load_hrly[s:e], label = 'Abs Chiller')
    plt.xlabel('Hours')
    plt.ylabel('Cooling Production [Tons]')
    plt.legend(handles = [line3])







    
    
    
    

#chp vs boiler fuel consumption
year_one_chp_fuel = api_response['outputs']['Scenario']['Site']['CHP']['year_one_fuel_used_mmbtu']
print("Annual CHP Fuel Use =", '%.3f' %year_one_chp_fuel)
year_one_boiler_fuel = api_response['outputs']['Scenario']['Site']['Boiler']['year_one_boiler_fuel_consumption_mmbtu']
print("Annual Boiler Fuel Use =", '%.3f' %year_one_boiler_fuel)

#chp_annual_runhrs = np.count_nonzero(chp_elec_total_hourly)
chp_annual_runhrs = np.sum(chp_elec_total_hrly>1) #alternative to np.count_nonzero to eliminate very small numbers. Sums the number of TRUE
#chp_annual_runhrs = chp_elec_total_hourly[chp_elec_total_hourly>1].count() #alternative to np.count_nonzero to eliminate very small numbers
chp_elec_cf_when_on = chp_avg_load_when_on/chp_size_kwe
chp_annual_heatinghrs = np.sum(chp_therm_total_hrly>0.01) #alternative to np.count_nonzero to eliminate very small numbers
chp_capacity_elec_to_load_peak = chp_size_kwe/bau_grid_load_peak
chp_capacity_elec_to_load_mean = chp_size_kwe/bau_grid_load_mean
chp_capacity_therm_to_load_peak = chp_size_kwe*chp_therm_to_elec_full_load/(bau_therm_load_peak*1e6/3412)
chp_capacity_therm_to_load_mean = chp_size_kwe*chp_therm_to_elec_full_load/(bau_therm_load_mean*1e6/3412)

load_hours_less_than_chp_capacity = np.sum(bau_grid_load_hrly < chp_size_kwe)

#find base load of bau_grid_load_hrly. Using value with 85% exceedance as described in ASHRAE 'Combined Heat and Power Design Guide' 
base_load_site_elec = bau_grid_load_hrly.sort_values(0, ascending = False).iloc[int(8760*.85)-1]
base_load_site_therm = bau_boiler_load_fuel_hrly.sort_values(0, ascending = False).iloc[int(8760*.85)-1]/boiler_eff
chp_capacity_elec_to_base_load = chp_size_kwe/base_load_site_elec
chp_capacity_therm_to_base_load = chp_size_kwe*chp_therm_to_elec_full_load/(base_load_site_therm*1e6/3412)
chp_gen_elec_to_load = chp_elec_total_hrly.resample('Y').sum()/bau_grid_load_hrly.resample('Y').sum()
chp_gen_therm_to_load = chp_therm_total_hrly.resample('Y').sum()/(bau_boiler_load_fuel_hrly.resample('Y').sum()*boiler_eff)
#net_fuel_consumption_change = 

print("Annual CHP Electric Efficiency =", '%.3f' %chp_elec_effic)
print("Annual CHP Total Efficiency =", '%.3f' %chp_total_effic)
print("Annual CHP Elec Capacity Factor =", '%.2f' %chp_elec_cf)
print("Annual CHP Elec Capacity Factor when running =", '%.2f' %chp_elec_cf_when_on)
print("Annual CHP run hours =", '%.0f' %chp_annual_runhrs, "(fraction",'%.2f'%(chp_annual_runhrs/8760), ")")
print("Annual CHP hours of useful heat =", '%.0f' %chp_annual_heatinghrs, "(fraction",'%.2f'%(chp_annual_heatinghrs/8760), ")")
print("Annual CHP mean power output when on =", '%.0f' %chp_avg_load_when_on, "kW")
print("Elec peak load =", '%.0f' %bau_grid_load_peak, "kW")
print("Elec mean load =", '%.0f' %bau_grid_load_mean, "kW")
print("Elec base load =", '%.0f' %base_load_site_elec, "kW (85% exceedance value)")
print("Heating peak load =", '%.2f' %bau_therm_load_peak, "MMBtu")
print("Heating mean load =", '%.2f' %bau_therm_load_mean, "MMBtu")
print("Heating base load =", '%.2f' %base_load_site_therm, "MMBtu (85% exceedance value)")
print("Ratio of CHP elec capacity to peak elec load =", '%.3f' %chp_capacity_elec_to_load_peak)
print("Ratio of CHP elec capacity to mean elec load =", '%.3f' %chp_capacity_elec_to_load_mean)
print("Ratio of CHP elec capacity to base elec load =", '%.2f' %chp_capacity_elec_to_base_load)
print("Ratio of CHP therm capacity to peak heat load =", '%.3f' %chp_capacity_therm_to_load_peak)
print("Ratio of CHP therm capacity to mean heat load =", '%.3f' %chp_capacity_therm_to_load_mean)
print("Ratio of CHP therm capacity to base heat load =", '%.3f' %chp_capacity_therm_to_base_load)
print("Useful heat recovery ratio =", '%.3f' )
print("Annual fraction of site elec load served by CHP =", '%.3f' %chp_gen_elec_to_load.iloc[0][0])
print("Annual fraction of site thermal load served by CHP =", '%.3f' %chp_gen_therm_to_load.iloc[0][0])


# Post 2

In [ ]:
api_response = reo_optimize_localhost(post_2)

## Start looking at the response/results data

In [ ]:
api_response['messages']

In [ ]:
type(post_2)

In [ ]:
api_response['outputs']['Scenario']['Site']['CHP']['year_one_thermal_to_load_series_mmbtu_per_hour']

## Save API response and scenario data into a .pickle file for later use

In [ ]:
save_directory = 'saved_results/'
pickle_file = 'pickle_file_test_1'
with open(save_directory + pickle_file + '.pickle', 'wb') as handle:
    data = {'API_response': api_response, 'scenario_data': scenario_data}
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Extract relevant data and assign object names to them

In [ ]:
CHP_max_heat_full_load

In [ ]:
#Pull CHP parameter inputs and do intermediate calcs for performance checks
from scipy.stats import linregress
chp_elec_eff_full_load = api_response['inputs']['Scenario']['Site']['CHP']['elec_effic_full_load']
chp_elec_eff_half_load = api_response['inputs']['Scenario']['Site']['CHP']['elec_effic_half_load']
chp_therm_eff_full_load = api_response['inputs']['Scenario']['Site']['CHP']['thermal_effic_full_load']
chp_therm_eff_half_load = api_response['inputs']['Scenario']['Site']['CHP']['thermal_effic_half_load']
chp_size_kwe = api_response['outputs']['Scenario']['Site']['CHP']['size_kw']
CHP_fuel_burn_full_load = chp_size_kwe / chp_elec_eff_full_load * 3412/1e6 #[MMBtu/hr]
CHP_fuel_burn_half_load = 0.5*chp_size_kwe / chp_elec_eff_half_load * 3412/1e6
slope, intercept, rvalue, pvalue, stderr = linregress([0.5*chp_size_kwe, chp_size_kwe], [CHP_fuel_burn_half_load, CHP_fuel_burn_full_load])
CHP_elec_slope = slope
CHP_elec_intercept = intercept
CHP_max_heat_full_load = CHP_fuel_burn_full_load * chp_therm_eff_full_load #[MMBtu/hr]
CHP_max_heat_half_load = CHP_fuel_burn_half_load * chp_therm_eff_half_load 
slope, intercept, rvalue, pvalue, stderr = linregress([0.5*chp_size_kwe, chp_size_kwe], [CHP_max_heat_half_load, CHP_max_heat_full_load])
CHP_therm_slope = slope
CHP_therm_intercept = intercept
print(CHP_elec_slope)
print(CHP_elec_intercept)
print(CHP_therm_slope)
print(CHP_therm_intercept)

In [ ]:
#Olis
load_year = str(api_response['inputs']['Scenario']['Site']['LoadProfile']['year'])
dti = pd.date_range( load_year +'-01-01', periods=8760, freq='H')
bau_grid_load_less_chiller_hrly = pd.DataFrame(api_response['inputs']['Scenario']['Site']
                                               ['LoadProfile']['loads_kw'], index = dti)
bau_boiler_load_fuel_hrly = pd.DataFrame(api_response['inputs']['Scenario']['Site']
                                         ['LoadProfileBoilerFuel']['loads_mmbtu_per_hour'], index = dti)
bau_chiller_load_elec_hrly = pd.DataFrame(api_response['inputs']['Scenario']['Site']
                                          ['LoadProfileChillerElectric']['loads_kw'], index = dti)
boiler_eff = api_response['inputs']['Scenario']['Site']['Boiler']['boiler_efficiency']

bau_grid_load_hrly = bau_grid_load_less_chiller_hrly + bau_chiller_load_elec_hrly
bau_grid_load_peak = np.max(bau_grid_load_hrly)
bau_grid_load_mean = np.mean(bau_grid_load_hrly)
bau_grid_load_total = np.sum(bau_grid_load_hrly)
bau_therm_load_peak = np.max(bau_boiler_load_fuel_hrly)*boiler_eff #converted units of fuel to units of heat
bau_therm_load_mean = np.mean(bau_boiler_load_fuel_hrly)*boiler_eff
bau_therm_load_total = np.sum(bau_boiler_load_fuel_hrly)*boiler_eff
chp_therm_to_elec_full_load = chp_therm_eff_full_load/chp_elec_eff_full_load


chp_elec_total_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                   ['year_one_electric_production_series_kw'], index = dti)
chp_therm_to_load_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                      ['year_one_thermal_to_load_series_mmbtu_per_hour'], index = dti)
chp_therm_to_TES_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['CHP']
                                     ['year_one_thermal_to_tes_series_mmbtu_per_hour'], index = dti)
chp_therm_total_hrly = chp_therm_to_load_hrly + chp_therm_to_TES_hrly
chp_fuel_annual = api_response['outputs']['Scenario']['Site']['CHP']['year_one_fuel_used_mmbtu']
chp_elec_effic = np.sum(chp_elec_total_hrly)/(chp_fuel_annual * 1E6/3412.0)
chp_total_effic = (np.sum(chp_elec_total_hrly) + np.sum(chp_therm_total_hrly)* 1E6/3412.0)/(chp_fuel_annual * 1E6/3412.0)
chp_elec_cf = np.sum(chp_elec_total_hrly)/(chp_size_kwe*8760)
chp_avg_load_when_on = chp_elec_total_hrly[chp_elec_total_hrly>1].mean()
boiler_fuel_hrly = pd.DataFrame(api_response['outputs']['Scenario']['Site']['Boiler']
                                ['year_one_boiler_fuel_consumption_series_mmbtu_per_hr'], index = dti)
boiler_fuel_annual = boiler_fuel_hrly.resample('Y').sum()

#chp_annual_runhrs = np.count_nonzero(chp_elec_total_hourly)
chp_annual_runhrs = np.sum(chp_elec_total_hrly>1) #alternative to np.count_nonzero to eliminate very small numbers. Sums the number of TRUE
#chp_annual_runhrs = chp_elec_total_hourly[chp_elec_total_hourly>1].count() #alternative to np.count_nonzero to eliminate very small numbers
chp_elec_cf_when_on = chp_avg_load_when_on/chp_size_kwe
chp_annual_heatinghrs = np.sum(chp_therm_total_hrly>0.01) #alternative to np.count_nonzero to eliminate very small numbers
chp_capacity_elec_to_load_peak = chp_size_kwe/bau_grid_load_peak
chp_capacity_elec_to_load_mean = chp_size_kwe/bau_grid_load_mean
chp_capacity_therm_to_load_peak = chp_size_kwe*chp_therm_to_elec_full_load/(bau_therm_load_peak*1e6/3412)
chp_capacity_therm_to_load_mean = chp_size_kwe*chp_therm_to_elec_full_load/(bau_therm_load_mean*1e6/3412)

load_hours_less_than_chp_capacity = np.sum(bau_grid_load_hrly < chp_size_kwe)

#find base load of bau_grid_load_hrly. Using value with 85% exceedance as described in ASHRAE 'Combined Heat and Power Design Guide' 
base_load_site_elec = bau_grid_load_hrly.sort_values(0, ascending = False).iloc[int(8760*.85)-1]
base_load_site_therm = bau_boiler_load_fuel_hrly.sort_values(0, ascending = False).iloc[int(8760*.85)-1]/boiler_eff
chp_capacity_elec_to_base_load = chp_size_kwe/base_load_site_elec
chp_capacity_therm_to_base_load = chp_size_kwe*chp_therm_to_elec_full_load/(base_load_site_therm*1e6/3412)
chp_gen_elec_to_load = chp_elec_total_hrly.resample('Y').sum()/bau_grid_load_hrly.resample('Y').sum()
chp_gen_therm_to_load = chp_therm_total_hrly.resample('Y').sum()/(bau_boiler_load_fuel_hrly.resample('Y').sum()*boiler_eff)
#net_fuel_consumption_change = 

print("Annual CHP Electric Efficiency =", '%.3f' %chp_elec_effic)
print("Annual CHP Total Efficiency =", '%.3f' %chp_total_effic)
print("Annual CHP Elec Capacity Factor =", '%.2f' %chp_elec_cf)
print("Annual CHP Elec Capacity Factor when running =", '%.2f' %chp_elec_cf_when_on)
print("Annual CHP run hours =", '%.0f' %chp_annual_runhrs, "(fraction",'%.2f'%(chp_annual_runhrs/8760), ")")
print("Annual CHP hours of useful heat =", '%.0f' %chp_annual_heatinghrs, "(fraction",'%.2f'%(chp_annual_heatinghrs/8760), ")")
print("Annual CHP mean power output when on =", '%.0f' %chp_avg_load_when_on, "kW")
print("Elec peak load =", '%.0f' %bau_grid_load_peak, "kW")
print("Elec mean load =", '%.0f' %bau_grid_load_mean, "kW")
print("Elec base load =", '%.0f' %base_load_site_elec, "kW (85% exceedance value)")
print("Heating peak load =", '%.2f' %bau_therm_load_peak, "MMBtu")
print("Heating mean load =", '%.2f' %bau_therm_load_mean, "MMBtu")
print("Heating base load =", '%.2f' %base_load_site_therm, "MMBtu (85% exceedance value)")
print("Ratio of CHP elec capacity to peak elec load =", '%.3f' %chp_capacity_elec_to_load_peak)
print("Ratio of CHP elec capacity to mean elec load =", '%.3f' %chp_capacity_elec_to_load_mean)
print("Ratio of CHP elec capacity to base elec load =", '%.2f' %chp_capacity_elec_to_base_load)
print("Ratio of CHP therm capacity to peak heat load =", '%.3f' %chp_capacity_therm_to_load_peak)
print("Ratio of CHP therm capacity to mean heat load =", '%.3f' %chp_capacity_therm_to_load_mean)
print("Ratio of CHP therm capacity to base heat load =", '%.3f' %chp_capacity_therm_to_base_load)
print("Useful heat recovery ratio =", '%.3f' )
print("Annual fraction of site elec load served by CHP =", '%.3f' %chp_gen_elec_to_load.iloc[0][0])
print("Annual fraction of site thermal load served by CHP =", '%.3f' %chp_gen_therm_to_load.iloc[0][0])


In [ ]:
chp_gen_elec_to_load_monthly = chp_elec_total_hrly.resample('M').sum()/bau_grid_load_hrly.resample('M').sum()
chp_gen_therm_to_load_monthly = chp_therm_to_load_hrly.resample('M').sum()/(bau_boiler_load_fuel_hrly.resample('M').sum()*boiler_eff)
blankdata = []*12
dfmonthly = pd.DataFrame(blankdata)
dfmonthly = pd.concat([chp_gen_elec_to_load_monthly, chp_gen_therm_to_load_monthly], axis = 1)
dfmonthly.columns = ['CHP Elec fraction', 'CHP Heat fraction']
print(dfmonthly)

In [ ]:
#check for excess heat to heating load in every hour
heat_balance = boiler_eff*bau_boiler_load_fuel_hrly-chp_therm_to_load_hrly

In [ ]:
TimeSteps = 8760
# Names of techs could have "NM" appended, depending on the analysis
GridName = 'UTIL1'
BoilerName = 'BOILER'
PVName = 'PV'
CHPName = 'CHP'
ElecChlName = 'ELECCHL'
AbsChlName = 'ABSORPCHL'
NumLoads = 5  # 1R, 1W, 1X, 1S, T

# Create a nice way to chain filters by using this custom function and Pandas mask method
def mask(df, key, value):
    return df[df[key] == value]
pd.DataFrame.mask = mask

# Electric load and supply
try:
    grid_num = scenario_data['Inputs/']['constant_' + run_uuid + '.dat'].mask('Tech', GridName).index.item()
    bau_grid_load_elec_hourly_kwe = scenario_data['Inputs/']['Load8760_' + run_uuid + '.dat'].astype('float64', errors='ignore')
    bau_grid_load_elec_total_kwhe = scenario_data['Inputs/']['LoadSize_' + run_uuid + '.dat'].astype('float64', errors='ignore')
    
    elec_price_hourly = scenario_data['Outputs_bau/']['energy_cost.txt'].astype('float64', errors='ignore')
    demand_charge_hourly = scenario_data['Outputs_bau/']['demand_cost.txt'].astype('float64', errors='ignore')
    grid_hourly_kwe = scenario_data['Outputs/']['GridToLoad.csv'].astype('float64', errors='ignore')
    total_grid_kwhe = np.sum(grid_hourly_kwe)
except:
    print("No electric load or supply in this analysis")

# Boiler fuel load and supply
try:
    # Units for hot-thermal and fuel power are mmbtu/hr and energy is mmbtu - did not append units to names for brevity
    boiler_num = scenario_data['Inputs/']['constant_' + run_uuid + '.dat'].mask('Tech', BoilerName).index.item()
    boiler_effic = scenario_data['Inputs/']['Utility/']['FuelBurnRate.dat'].astype('float64', errors='ignore')['BoilerEfficiency'][0]
    bau_boiler_load_fuel_hourly = scenario_data['Inputs/']['Load8760BoilerFuel_' + run_uuid + '.dat'].astype('float64', errors='ignore')
    bau_boiler_load_fuel_total = scenario_data['Inputs/']['LoadSizeBoilerFuel_' + run_uuid + '.dat'].astype('float64', errors='ignore')
    boiler_fuel_rate_hourly = scenario_data['Inputs/']['Utility/']['FuelCost.dat']['FuelRate'].iloc[boiler_num* \
                                                                                TimeSteps:(boiler_num + 1)*TimeSteps]
    boiler_fuel_hourly = scenario_data['Outputs/']['FuelToBoiler.csv'].astype('float64', errors='ignore')
    boiler_thermal_hourly = scenario_data['Outputs/']['BoilerThermal.csv'].astype('float64', errors='ignore')
    total_boiler_fuel = np.sum(boiler_fuel_hourly)
except:
    print("No hot thermal/fuel load or boiler supply in this analysis")
    
# CHP electric and hot thermal production
# TODO look for files containing certain strings to assign only relevant variables because few/no irrelevant ones will be writting from Xpress
try:
    # Units for hot-thermal and fuel power are mmbtu/hr and energy is mmbtu - did not append units to names for brevity
    chp_num = scenario_data['Inputs/']['constant_' + run_uuid + '.dat'].mask('Tech', CHPName).index.item()
    chp_size_kwe = scenario_data['Outputs/']['REopt_results.json']['chp_kw']
    chp_fuel_rate_hourly = scenario_data['Inputs/']['Utility/']['FuelCost.dat']['FuelRate'].iloc[chp_num* \
                                                                        TimeSteps:(chp_num + 1)*TimeSteps]
    chp_elec_to_load_hourly_kwe = scenario_data['Outputs/']['CHPtoLoad.csv'].astype('float64', errors='ignore')
    chp_elec_to_grid_hourly_kwe = scenario_data['Outputs/']['CHPtoGrid.csv'].astype('float64', errors='ignore') # This is 1W + 1R
    chp_elec_total_hourly = scenario_data['Outputs/']['CHPElectricProduction.csv'].astype('float64', errors='ignore')
    # Calculate thermal production total, which may be above Thermal-to-Load because of curtailment/above-load
    chp_thermalprod_slope = scenario_data['Inputs/']['CHPPerformance_' + run_uuid + '.dat'].astype('float64', errors='ignore')['CHPThermalProdSlope'][0]
    chp_thermalprod_intercept = scenario_data['Inputs/']['CHPPerformance_' + run_uuid + '.dat'].astype('float64', errors='ignore')['CHPThermalProdIntercept'][0]
    chp_thermal_to_load_hourly = scenario_data['Outputs/']['CHPThermaltoLoad.csv'].astype('float64', errors='ignore')
    chp_thermal_max_production_calculated_hourly = chp_thermalprod_intercept * chp_size_kwe * (chp_elec_total_hourly > 0) + \
                                                    chp_thermalprod_slope * chp_elec_total_hourly
    # TBD Calculate fuel used by timestep, manually because not by timestep in Xpress
    chp_fuelburnrate_slope = scenario_data['Inputs/']['Utility/']['FuelBurnRate.dat'].astype('float64', errors='ignore')[chp_num*NumLoads:(chp_num+1)*NumLoads].iloc[0,:]['FuelBurnRateM']
    chp_fuelburnrate_intercept = scenario_data['Inputs/']['Utility/']['FuelBurnRate.dat'].astype('float64', errors='ignore')[chp_num*NumLoads:(chp_num+1)*NumLoads].iloc[0,:]['FuelBurnRateB']
    chp_fuelburned = chp_fuelburnrate_intercept * chp_size_kwe  * (chp_elec_total_hourly > 0) + \
                                                    chp_fuelburnrate_slope * chp_elec_total_hourly
except:
    print("No CHP (chosen) in this analysis")    
    
# TODO: PV, cold electric load, absorption chiller, electric chiller, battery, hot TES, cold TES

# Some useful things for indexing on techs in the .dat files
#df = df.replace(np.nan, 0, regex=True)  # For replacing NaN with 0, needed for some variables
#PV_num = scenario_data['Inputs/']['constant_' + run_uuid + '.dat'].mask('Tech', PVName).index.item()
#ElecChl_num = scenario_data['Inputs/']['constant_' + run_uuid + '.dat'].mask('Tech', ElecChlName).index.item()
#AbsChl_num = scenario_data['Inputs/']['constant_' + run_uuid + '.dat'].mask('Tech', AbsChlName).index.item()
    
#dispatch_CHP_Elec_To_Batt = scenario_data['Outputs/']['CHPtoBatt.csv'].astype('float64', errors='ignore')

# LEGACY FROM DESKTOP DATA ANALYSIS
# Extract storage dispatch
#storage_charge = scenario_data['ElecToStore.csv'].copy() * -1
#storage_discharge = scenario_data['ElecFromStore.csv'].copy()
#storage_dispatch = storage_charge + storage_discharge
#EtaStor = -1 * np.sum(storage_discharge.values) / np.sum(storage_charge.values)
    
# Extract storage dispatch
#storage_charge = scenario_data['ElecToStore.csv'].copy() * -1
#storage_discharge = scenario_data['ElecFromStore.csv'].copy()
#storage_dispatch = storage_charge + storage_discharge
#SOC = scenario_data['StoredEnergy.csv'].copy()

## Calculations and printing results for validation

In [ ]:
print("Total CHP Electric Production (kWh) = ", '%.2f' %np.sum(chp_elec_total_hrly.values))
print("Total CHP Thermal Production (MMBtu) = ", '%.2f' %np.sum(chp_thermal_to_load_hrly.values))
print("Total CHP Fuel Used (MMBtu) = ", '%.2f' %np.sum(chp_fuelburned.values))
chp_elec_effic = np.sum(chp_elec_total_hourly.values)/(np.sum(chp_fuelburned.values) * 1E6/3412.0)
chp_full_thermal_effic = np.sum(chp_thermal_max_production_calculated_hourly.values) / np.sum(chp_fuelburned.values)
chp_actual_thermal_effic = np.sum(chp_thermal_to_load_hourly.values) / np.sum(chp_fuelburned.values)
print("Average CHP Electric Efficiency = ", '%.3f' %chp_elec_effic)
print("Average Potential (all thermal used) CHP Thermal Efficiency = ", '%.3f' %chp_full_thermal_effic)
print("Average Actual CHP Thermal Efficiency = ", '%.3f' %chp_actual_thermal_effic)
chp_total_effic = np.sum(chp_elec_total_hourly.values + chp_thermal_to_load_hourly.values * 1E6/3412.0)/(np.sum(chp_fuelburned.values) * 1E6/3412.0)
print("Average CHP Elec + Thermal Efficiency = ", '%.3f' %chp_total_effic)

In [ ]:

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.plot(chp_elec_total_hrly.values, markersize=1, linestyle='', marker='o', color='b') #.values turns it into an numpy array
plt.plot()
#plt.plot(bau_chiller_load_elec_hourly.values)
#plt.plot(bau_grid_load)
#plt.xlim(0,200)

## Load in REopt Desktop results

In [ ]:
file_directory = r'C:\Bill\REopt\Output\TempResults'
results_dict, filepath_list = REopt_TempResults(file_directory)
# Check the high level results
results_dict['REOresults.csv']

## Extract relevant data from Desktop REopt to compare with API

In [ ]:
PVName = 'PVNM'
CHPName = 'CHPNM'

# Extract baseline load data
load_hourly_baseline = results_dict['Load.csv'].copy()
try:
    load_elec_hourly_baseline = load_hourly_baseline.iloc[:, 0]
    load_fuel_hourly_baseline = load_hourly_baseline.iloc[:, 1]  # I had to add fuel (e.g. "2R") load to Load.csv in Xpress
    load_elec_total_baseline = np.sum(load_elec_hourly_baseline)
    load_fuel_total_baseline = np.sum(load_fuel_hourly_baseline)
    elec_price_hourly = results_dict['ElecRateHourly.csv'].copy().iloc[:, 0]
    demand_charge_hourly = results_dict['ElecRateHourly.csv'].copy().iloc[:, 1]
except:
    print("Either no elec or thermal load in this analysis")

# Extract dispatch results data
dispatch_hourly = results_dict['DispatchHourly.csv'].copy()
# Using the Pandas 'mask', as defined above, filter to get specific data
try:
    dispatch_elec_hourly_CHP_1R = dispatch_hourly.mask('Tech', CHPName).mask('FuelBin', '1R')
    dispatch_elec_hourly_CHP_1W = dispatch_hourly.mask('Tech', CHPName).mask('FuelBin', '1W')
    dispatch_elec_hourly_CHP_1X = dispatch_hourly.mask('Tech', CHPName).mask('FuelBin', '1X')
    dispatch_elec_hourly_CHP_1S = dispatch_hourly.mask('Tech', CHPName).mask('FuelBin', '1S')
    dispatch_heat_hourly_CHP = dispatch_hourly.mask('Tech', CHPName).mask('FuelBin', '2R')
    FuelUsed_hourly_CHP = results_dict['FuelUsed.csv'].copy().iloc[:, 2]
    dispatch_elec_total_CHP = np.sum(dispatch_elec_hourly_CHP_1R['PF*dvRP']+dispatch_elec_hourly_CHP_1W['PF*dvRP'])
    dispatch_heat_total_CHP = np.sum(dispatch_heat_hourly_CHP['PF*dvRP'])
except:
    print("No CHPNM (chosen) in this analysis")

try:
    dispatch_elec_hourly_Util1 = dispatch_hourly.mask('Tech', 'UTIL1').mask('FuelBin', '1R')
    dispatch_heat_hourly_Util2 = dispatch_hourly.mask('Tech', 'UTIL2').mask('FuelBin', '2R')
    dispatch_elec_total_Util1 = np.sum(dispatch_elec_hourly_Util1['PF*dvRP'])
    dispatch_heat_total_Util2 = np.sum(dispatch_heat_hourly_Util2['PF*dvRP'])
except:
    print("Either UTIL1 or UTIL2 not used in this analysis")

try:
    dispatch_elec_hourly_PV_1R = dispatch_hourly.mask('Tech', PVName).mask('FuelBin', '1R')
    dispatch_elec_hourly_PV_1W = dispatch_hourly.mask('Tech', PVName).mask('FuelBin', '1W')
    dispatch_elec_hourly_PV_1X = dispatch_hourly.mask('Tech', PVName).mask('FuelBin', '1X')
    dispatch_elec_hourly_PV_1S = dispatch_hourly.mask('Tech', PVName).mask('FuelBin', '1S')
except:
    print("No PV in this analysis")
    
# Extract storage dispatch
storage_charge = results_dict['ElecToStore.csv'].copy() * -1
storage_discharge = results_dict['ElecFromStore.csv'].copy()
storage_dispatch = storage_charge + storage_discharge
EtaStor = -1 * np.sum(storage_discharge.values) / np.sum(storage_charge.values)
    
# Extract storage dispatch
storage_charge = results_dict['ElecToStore.csv'].copy() * -1
storage_discharge = results_dict['ElecFromStore.csv'].copy()
storage_dispatch = storage_charge + storage_discharge
SOC = results_dict['StoredEnergy.csv'].copy()

## ===============================================================

## Example from Josiah for how to setup batch API calls

## Comparing DOE Reference Buildings

In [ ]:
doe_list = ['RetailStore', 'StripMall', 'Warehouse', 'Supermarket']

In [ ]:
api_response_dict = dict()

for name in doe_list:
    newpost = post
    newpost['Scenario']['Site']["LoadProfile"]["doe_reference_name"] = name
    
    print('Beginning Scenario:', name)
    api_response = reo_optimize_localhost(newpost)
    api_response_dict[name] = api_response
    
    if api_response['outputs']['Scenario']['status'] != 'optimal':
        print('non-optimial solution')

In [ ]:
results_dict['RetailStore']['outputs']

In [ ]:
results_plots(results_dict)

# Try to loop through cities

In [ ]:
# SF, PHX, CHI, NYC, DEN
location = ['SF', 'PHX', 'CHI', 'NYC', 'DEN']
lat = [37.78, 33.45, 41.983, 40.65, 39.742]
long = [-122.45, -111.983, -87.917, -73.8, -105.179]
elevation = [0, 1106, 659, 16, 6000]
T_SF = list(pd.read_csv("SanFran_Tamb_degF.csv", squeeze=True, header = None))
T_PHX = list(pd.read_csv("PHX_Tamb_degF.csv", squeeze=True, header = None)) 
T_CHI = list(pd.read_csv("CHI_Tamb_degF.csv", squeeze=True, header = None))
T_NYC = list(pd.read_csv("NYC_Tamb_degF.csv", squeeze=True, header = None))
T_DEN = list(pd.read_csv("Golden_Tamb_degF.csv", squeeze=True, header = None))
T_amb = [T_SF, T_PHX, T_CHI, T_NYC, T_DEN]

#------if forcing a size------
    #largehotel - sf elec rate - sf commercial ng price - ICE-PV
#CHP_base_size = [274.9, 323.4, 366.4, 367.3, 352] 
#PV_base_size = [193.3, 495.7, 161, 228.2, 156.7]
    #hospital - sf elec rate - sf industrial ng price - ICE-PV
#CHP_base_size = [740.7, 824.2, 1004.2, 1014, 945.6] 
#PV_base_size = [1822.8, 1950.9, 1338.8, 1358.7, 1318.6]
    #hospital - sf elec rate - sf industrial ng price - MT-PV-BESS
#CHP_base_size = [334.1, 291.2, 430, 407.6, 380.7] 
#PV_base_size = [2283.3, 2878.3, 2475.4, 2764.2, 2637.2]
#BESS_base_power = [340.8, 530.5, 379.9, 376.9, 441.2]
#BESS_base_capacity = [1353.1, 1621.3, 1262, 1295.9, 1670.3]
    #largehotel - sf elec rate - sf industrial ng price - MT-PV-BESS
CHP_base_size = [129.2, 72, 188, 187.9, 190.8] 
PV_base_size = [404.9, 978.9, 448.8, 518.7, 429.6]
BESS_base_power = [182.6, 284.5, 168.7, 161.8, 172.6]
BESS_base_capacity = [617.7, 1410.9, 696.9, 729.1, 803.3]


api_response_dict = dict()

mode_force = 2  # 1 = fix size   # 2 = do not fix size
mode_battery = 2 # 1 = fix battery size # 2 = do not fix battery size
for i in range(5):
    newpost = post_1
    newpost['Scenario']['Site']["latitude"] = lat[i]
    newpost['Scenario']['Site']["longitude"] = long[i]
    newpost['Scenario']['Site']["outdoor_air_temp_degF"] = T_amb[i]
    newpost['Scenario']['Site']["elevation_ft"] = elevation[i]
    
    #if forcing size
    if mode_force == 1:
        newpost['Scenario']['Site']['CHP']['min_kw'] = CHP_base_size[i]
        newpost['Scenario']['Site']['CHP']['max_kw'] = CHP_base_size[i]
        newpost['Scenario']['Site']['PV']['min_kw'] = PV_base_size[i]
        newpost['Scenario']['Site']['PV']['max_kw'] = PV_base_size[i]
        if mode_battery == 1:
            newpost['Scenario']['Site']['Storage']['min_kw'] = BESS_base_power[i]
            newpost['Scenario']['Site']['Storage']['max_kw'] = BESS_base_power[i]
            newpost['Scenario']['Site']['Storage']['min_kwh'] = BESS_base_capacity[i]
            newpost['Scenario']['Site']['Storage']['max_kwh'] = BESS_base_capacity[i]
        
    
    #print('Beginning Scenario:', name)
    api_response = reo_optimize_localhost(newpost)
    api_response_dict[i] = api_response

In [ ]:
save_directory = 'saved_results_3/'
pickle_file = 'DEN_MT-PV-BESS_corr'
with open(save_directory + pickle_file + '.pickle', 'wb') as handle:
    data = {'API_response': api_response_dict[4], 'scenario_data': post_1} #post data will be off for each case, because each post is within the loop
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
x=3
print(api_response_dict[x]['outputs']['Scenario']['Site']['CHP']['size_kw'])
print(api_response_dict[x]['outputs']['Scenario']['Site']['PV']['size_kw'])
print(api_response_dict[x]['outputs']['Scenario']['Site']['Storage']['size_kw'])
print(api_response_dict[x]['outputs']['Scenario']['Site']['Storage']['size_kwh'])

print(api_response_dict[x]['outputs']['Scenario']['Site']['Financial']['lcc_us_dollars'])
print(api_response_dict[x]['outputs']['Scenario']['Site']['Financial']['lcc_bau_us_dollars'])
print(api_response_dict[x]['outputs']['Scenario']['Site']['Financial']['npv_us_dollars'])